Practicumopdrachten Week 4.3 (eerste kans)

In [1]:
import pandas as pd
import pyodbc
import sqlite3
import numpy as np
import re
import sys

# Mssql database
Mssql server database

In [2]:
# database name
DB = {
    'servername': '(local)\\SQLEXPRESS',
    'database': 'DEDS_DataWarehouse',}

## Database aanmaken

In [3]:
# Connect to the SQL Server database, not a specific database but the master
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + ';DATABASE=master;Trusted_Connection=yes')

# Create a cursor object
cur = conn.cursor()

try:
    # Kill all active connections to the database
    cur.execute("""
        DECLARE @kill varchar(8000) = '';
        SELECT @kill = @kill + 'kill ' + convert(varchar, spid) + ';'
        FROM master.dbo.sysprocesses
        WHERE dbid = db_id('{database}')
        AND spid <> @@spid;
        EXEC(@kill);
    """.format(database=DB['database']))
    conn.commit()

    # Close the current connection
    cur.close()
    conn.close()

    # Reconnect to the SQL Server database
    conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + ';DATABASE=master;Trusted_Connection=yes')
    cur = conn.cursor()

    # Read the SQL script file
    with open('DEDS_DataWarehouse_Creation_Script.sql', 'r') as file:
        sql_script = file.read()

    # Split the SQL script into individual statements
    sql_statements = re.split(r'GO\n', sql_script)

    # Execute the DROP DATABASE statement
    drop_database_stmt = sql_statements[0]
    cur.execute(drop_database_stmt)

    # Execute the CREATE DATABASE statement
    create_database_stmt = sql_statements[1]
    cur.execute(create_database_stmt)

    # Start a new transaction
    conn.autocommit = False

    # Execute the remaining SQL statements within the transaction
    for sql_statement in sql_statements[2:]:
        cur.execute(sql_statement)

    # Commit the transaction
    conn.commit()

except pyodbc.Error as e:
    print(f"Error: {e}")
    conn.rollback()
    sys.exit(1)

finally:
    # Close the cursor
    cur.close()

    # Close the connection
    conn.close()

## Database connectie

In [4]:
export_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')


# Create a cursor from the connection
export_cursor = export_conn.cursor()

# check if connection is successful, else throw an error
if export_conn:
    print("Connection with database is established")
    # setting the autocommit to True
    export_conn.autocommit = True
else:
    print("Connection with database is not established")
    raise Exception("Connection with database is not established")


Connection with database is established


# Brontabellen
importeer de brontabellen uit de brondata folder

In [5]:
# Connectie met sqlite databases
go_crm_connection = sqlite3.connect('data/raw/go_crm.sqlite')
go_sales_connection = sqlite3.connect('data/raw/go_sales.sqlite')
go_staff_connection = sqlite3.connect('data/raw/go_staff.sqlite')


In [6]:
# inlezen csv bestanden
go_sales_inventory_levels = pd.read_csv('data/raw/GO_SALES_INVENTORY_LEVELSData.csv')
go_sales_product_forcast = pd.read_csv('data/raw/GO_SALES_PRODUCT_FORECASTData.csv')

## Inlezen van de sqlite tabellen

### go_crm

In [7]:
go_crm_age_group = pd.read_sql_query("SELECT * FROM age_group", go_crm_connection)
go_crm_country = pd.read_sql_query("SELECT * FROM country", go_crm_connection)
go_crm_retailer = pd.read_sql_query("SELECT * FROM retailer", go_crm_connection)
go_crm_retailer_contact = pd.read_sql_query("SELECT * FROM retailer_contact", go_crm_connection)
go_crm_retailer_headquarters = pd.read_sql_query("SELECT * FROM retailer_headquarters", go_crm_connection)
go_crm_retailer_segment = pd.read_sql_query("SELECT * FROM retailer_segment", go_crm_connection)
go_crm_retailer_site = pd.read_sql_query("SELECT * FROM retailer_site", go_crm_connection)
go_crm_retailer_type = pd.read_sql_query("SELECT * FROM retailer_type", go_crm_connection)
go_crm_sales_demographic = pd.read_sql_query("SELECT * FROM sales_demographic", go_crm_connection)
go_crm_sales_territory = pd.read_sql_query("SELECT * FROM sales_territory", go_crm_connection)

### go_sales

In [8]:
go_sales_country = pd.read_sql_query("SELECT * FROM country", go_sales_connection)
go_sales_order_details = pd.read_sql_query("SELECT * FROM order_details", go_sales_connection)
go_sales_order_header = pd.read_sql_query("SELECT * FROM order_header", go_sales_connection)
go_sales_order_method = pd.read_sql_query("SELECT * FROM order_method", go_sales_connection)
go_sales_product = pd.read_sql_query("SELECT * FROM product", go_sales_connection)
go_sales_product_line = pd.read_sql_query("SELECT * FROM product_line", go_sales_connection)
go_sales_product_type = pd.read_sql_query("SELECT * FROM product_type", go_sales_connection)
go_sales_retailer_site = pd.read_sql_query("SELECT * FROM retailer_site", go_sales_connection)
go_sales_return_reason = pd.read_sql_query("SELECT * FROM return_reason", go_sales_connection)
go_sales_returned_item = pd.read_sql_query("SELECT * FROM returned_item", go_sales_connection)
go_sales_sales_branch = pd.read_sql_query("SELECT * FROM sales_branch", go_sales_connection)
go_sales_sales_staff = pd.read_sql_query("SELECT * FROM sales_staff", go_sales_connection)
go_sales_sales_target_data = pd.read_sql("SELECT * FROM SALES_TARGETData", go_sales_connection)

### go_staff

In [9]:
go_staff_course = pd.read_sql_query("SELECT * FROM course", go_staff_connection)
go_staff_sales_branch = pd.read_sql_query("SELECT * FROM sales_branch", go_staff_connection)
go_staff_sales_staff = pd.read_sql_query("SELECT * FROM sales_staff", go_staff_connection)
go_staff_satisfaction = pd.read_sql_query("SELECT * FROM satisfaction", go_staff_connection)
go_staff_satisfaction_type = pd.read_sql_query("SELECT * FROM satisfaction_type", go_staff_connection)
go_staff_training = pd.read_sql_query("SELECT * FROM training", go_staff_connection)

# Combineren van de tabellen
Combineren van de brondata om tot ons ETL te komen

| Returned_item                                  |
|------------------------------------------------|
| RETURNED_ITEM_code                             |
| RETURNED_ITEM_DATE_date                        |
| RETURNED_ITEM_QUANTITY_quantity                |
| RETURNED_ITEM_ORDER_DETAIL_CODE_detail_code    |
| RETURNED_ITEM_RETURN_REASON_code               |
| RETURNED_ITEM_RETURN_REASON_description_en     |
| RETURNED_ITEM_RETURNED_ITEMS_TOTAL_PRICE_total |

| Unit             |
|------------------|
| UNIT_id          |
| UNIT_COST_cost   |
| UNIT_PRICE_price |
| UNIT_SALE_sale   |

| Sales_staff                                          |
|------------------------------------------------------|
| SALES_STAFF__code                                    |
| SALES_STAFF_email                                    |
| SALES_STAFF_extension                                |
| SALES_STAFF_POSITION_EN_position                     |
| SALES_STAFF_WORK_PHONE_work_phone                    |
| SALES_STAFF_DATE_HIRED_hired                         |
| SALES_STAFF_MANAGER_CODE_manager                     |
| SALES_STAFF_FAX_fax                                  |
| SALES_STAFF_FIRST_NAME_first_name                    |
| SALES_STAFF_LAST_NAME_last_name                      |
| SALES_STAFF_FULL_NAME__full_name                     |
| SALES_STAFF_SALES_BRANCH_CODE_branch_code            |
| SALES_STAFF_SALES_BRANCH_ADDRESS_address             |
| SALES_STAFF_SALES_BRANCH_ADDRESS_EXTRA_address_extra |

| Satisfaction_type                   |
|-------------------------------------|
| SATISFACTION_TYPE_code              |
| SATISFACTION_TYPE_description       |
| SATISFACTION_TYPE_description_short |

| Course                   |
|--------------------------|
| COURSE_code              |
| COURSE_description       |
| COURSE_description_short |

| Year |
|------|
| YEAR |

| Date             | 
|------------------|
| DATE_date        |
| DATE_YEAR_year   |
| DATE_MONTH_month |
| DATE_DAY_day     |

| Order                               |
|-------------------------------------|
| ORDER_order_number                  |
| ORDER_ORDER_METHOD_CODE_method_code |
| ORDER_ORDER_METHOD_EN_method        |

| Retailer_site                             |
|-------------------------------------------|
| RETAILER_SITE_code                        |
| RETAILER_SITE_COUNTRY_CODE_country        |
| RETAILER_SITE_CITY_city                   |
| RETAILER_SITE_REGION_region               |
| RETAILER_SITE_POSTAL_ZONE_postal_zone     |
| RETAILER_SITE_RETAILER_CODE_retailer_code |
| RETAILER_SITE_ACTIVE_INDICATOR_indicator  |
| RETAILER_SITE_ADDRESS1_address            |
| RETAILER_SITE_ADDRESS2_address            |
| RETAILER_SITE_MAIN_ADDRESS__address       |

| Sales_branch                         |
|--------------------------------------|
| SALES_BRANCH_code                    |
| SALES_BRANCH_COUNTRY_CODE_country    |
| SALES_BRANCH_REGION_region           |
| SALES_BRANCH_CITY_city               |
| SALES_BRANCH_POSTAL_ZONE_postal_zone |
| SALES_BRANCH_ADDRESS1_address        |
| SALES_BRANCH_ADDRESS2_address        |
| SALES_BRANCH_MAIN_ADDRESS_address    |

| Retailer_contact                              |
|-----------------------------------------------|
| RETAILER_CONTACT_code                         |
| RETAILER_CONTACT_email                        |
| RETAILER_CONTACT_RETAILER_SITE_CODE_site_code |
| RETAILER_CONTACT_JOB_POSITION_EN_position     |
| RETAILER_CONTACT_EXTENSION_extension          |
| RETAILER_CONTACT_FAX_fax                      |
| RETAILER_CONTACT_GENDER_gender                |
| RETAILER_CONTACT_FIRST_NAME_first_name        |
| RETAILER_CONTACT_LAST_NAME_last_name          |
| RETAILER_CONTACT_FULL_NAME_full_name          |

| Retailer                                  |
|-------------------------------------------|
| RETAILER_code                             |
| RETAILER__name                            |
| RETAILER_COMPANY_CODEMR_company           |
| RETAILER_RETAILER_TYPE_retailer_type_code |
| RETAILER_RETAILER_TYPE_retailer_type_EN   |

| Product                              |
|--------------------------------------|
| PRODUCT_number                       |
| PRODUCT_name_product                 |
| PRODUCT_description_description      |
| PRODUCT_image_image                  |
| PRODUCT_INTRODUCTION_DATE_introduced |
| PRODUCT_PRODUCTION_COST_cost         |
| PRODUCT_MARGIN_margin                |
| PRODUCT_LANGUAGE_language            |
| PRODUCT_MINIMUM_SALE_PRICE_minPrice  |
| PRODUCT_PRODUCT_LINE_code            |
| PRODUCT_PRODUCT_LINE_code_en         |
| PRODUCT_PRODUCT_TYPE_code            |
| PRODUCT_PRODUCT_TYPE_code_en         |

| Order_details                        |
|--------------------------------------|
| ORDER_DETAILS_code                   |
| ORDER_DETAILS_QUANTITY_quantity      |
| ORDER_DETAILS_TOTAL_COST_total       |
| ORDER_DETAILS_TOTAL_MARGIN_margin    |
| ORDER_DETAILS_RETURN_CODE_returned   |
| ORDER_DETAILS_ORDER_NUMBER_order     |
| ORDER_DETAILS_PRODUCT_NUMBER_product |
| ORDER_DETAILS_UNIT_ID_unit           |

| SALES_TARGETData                  |
|-----------------------------------|
| SALES_TARGETDATA_id               |
| SALES_TARGETDATA_SALES_YEAR       |
| SALES_TARGETDATA_SALES_PERIOD     |
| SALES_TARGETDATA_RETAILER_NAME    |
| SALES_TARGETDATA_SALES_TARGET     |
| SALES_TARGETDATA_TARGET_COST      |
| SALES_TARGETDATA_TARGET_MARGIN    |
| SALES_TARGETDATA_SALES_STAFF_CODE |
| SALES_TARGETDATA_PRODUCT_NUMBER   |
| SALES_TARGETDATA_RETAILER_CODE    |

| Training                  |
|---------------------------|
| TRAINING_SALES_STAFF_CODE |
| TRAINING_COURSE_CODE      |
| TRAINING_YEAR             |

| Satisfaction                        |
|-------------------------------------|
| SATISFACTION_SALES_STAFF_CODE       |
| SATISFACTION_SATISFACTION_TYPE_CODE |
| SATISFACTION_YEAR                   |

| Order_header                       |
|------------------------------------|
| ORDER_HEADER_number                |
| ORDER_HEADER_RETAILER_NAME         |
| ORDER_HEADER_SALES_STAFF           |
| ORDER_HEADER_SALES_BRANCH_CODE     |
| ORDER_HEADER_ORDER_DATE            |
| ORDER_HEADER_RETAILER_SITE_CODE    |
| ORDER_HEADER_RETAILER_CONTACT_CODE |

| GO_SALES_INVENTORY_LEVELS                 |
|-------------------------------------------|
| GO_SALES_INVENTORY_LEVELS_id              |
| GO_SALES_INVENTORY_LEVELS_INVENTORY_COUNT |
| GO_SALES_INVENTORY_LEVELS_PRODUCT_NUMBER  |
| GO_SALES_INVENTORY_LEVELS_YEAR_MONTH      |

| GO_SALES_PRODUCT_FORECAST                 |
|-------------------------------------------|
| GO_SALES_PRODUCT_FORECAST_id              |
| GO_SALES_PRODUCT_FORECAST_EXPECTED_VOLUME |
| GO_SALES_PRODUCT_FORECAST_EXPECTED_COST   |
| GO_SALES_PRODUCT_FORECAST_EXPECTED_MARGIN |
| GO_SALES_PRODUCT_FORECAST_PRODUCT_NUMBER  |
| GO_SALES_PRODUCT_FORECAST_YEAR_MONTH      |

In [10]:
all_dataframes = []

### returned_item

In [11]:
# to be used in order details
returned_item_columns = ['RETURN_CODE', 'RETURN_DATE', 'RETURN_QUANTITY', 'ORDER_DETAIL_CODE', 'RETURN_REASON_CODE', 'RETURN_DESCRIPTION_EN']
returned_item = pd.merge(go_sales_returned_item, go_sales_return_reason, left_on='RETURN_REASON_CODE', right_on='RETURN_REASON_CODE')
returned_item = pd.merge(returned_item, go_sales_order_details, left_on='ORDER_DETAIL_CODE', right_on='ORDER_DETAIL_CODE')

# filtering the columns
returned_item = returned_item[returned_item_columns]

# renaming the columns
returned_item = returned_item.rename(columns={'RETURN_CODE': 'RETURNED_ITEM_code', 'RETURN_DATE': 'RETURNED_ITEM_DATE', 'RETURN_QUANTITY': 'RETURNED_ITEM_QUANTITY_quantity', 'ORDER_DETAIL_CODE': 'RETURNED_ITEM_ORDER_DETAIL_CODE_detail_code', 'RETURN_REASON_CODE': 'RETURNED_ITEM_RETURN_REASON_code', 'RETURN_DESCRIPTION_EN': 'RETURNED_ITEM_RETURN_REASON_description_en'})

all_dataframes.append(returned_item)

returned_item.head()

,RETURNED_ITEM_code,RETURNED_ITEM_DATE_date,RETURNED_ITEM_QUANTITY_quantity,RETURNED_ITEM_ORDER_DETAIL_CODE_detail_code,RETURNED_ITEM_RETURN_REASON_code,RETURNED_ITEM_RETURN_REASON_description_en
0,1491,1-8-2021 04:10:24,8,84858,5,Unsatisfactory product
1,1492,6-12-2020 18:46:19,2,84440,2,Incomplete product
2,1493,24-6-2021 09:23:14,22,84867,4,Wrong product shipped
3,1494,21-7-2021 00:00:09,20,84873,3,Wrong product ordered
4,1496,7-10-2020 05:13:58,2,84488,1,Defective product


### Unit

In [12]:
unit_columns = ['UNIT_COST', 'UNIT_PRICE', 'UNIT_SALE_PRICE', 'UNIT_id']
unit = go_sales_order_details

# make unit_id autoincrement
unit['UNIT_id'] = range(1, len(unit) + 1)

# filtering the columns
unit = unit[unit_columns]

# renaming the columns
unit = unit.rename(columns={'UNIT_COST': 'UNIT_COST_cost', 'UNIT_PRICE': 'UNIT_PRICE_price', 'UNIT_SALE_PRICE': 'UNIT_SALE_sale'})

all_dataframes.append(unit)

unit.head()

,UNIT_COST_cost,UNIT_PRICE_price,UNIT_SALE_sale,UNIT_id
0,16.1,22.54,22.54,1
1,16.1,22.54,22.54,2
2,16.1,22.54,22.54,3
3,16.1,22.54,22.54,4
4,16.1,22.54,22.54,5


### Sales_staff

In [13]:
sales_staff_columns = ['SALES_STAFF_CODE', 'EMAIL', 'EXTENSION', 'POSITION_EN', 'WORK_PHONE', 'DATE_HIRED', 'MANAGER_CODE', 'FAX', 'FIRST_NAME', 'LAST_NAME',  'ADDRESS1', 'ADDRESS2', 'SALES_BRANCH_CODE']

sales_staff = pd.merge(go_staff_sales_staff, go_staff_sales_branch, left_on='SALES_BRANCH_CODE', right_on='SALES_BRANCH_CODE')

# filtering the columns
sales_staff = sales_staff[sales_staff_columns]

# renaming the columns
sales_staff = sales_staff.rename(columns={'SALES_STAFF_CODE': 'SALES_STAFF_code', 'EMAIL': 'SALES_STAFF_email', 'EXTENSION': 'SALES_STAFF_extension', 'POSITION_EN': 'SALES_STAFF_POSITION_EN_position', 'WORK_PHONE': 'SALES_STAFF_WORK_PHONE_work_phone', 'DATE_HIRED': 'SALES_STAFF_DATE_HIRED_hired', 'MANAGER_CODE': 'SALES_STAFF_MANAGER_CODE_manager', 'FAX': 'SALES_STAFF_FAX_fax', 'FIRST_NAME': 'SALES_STAFF_FIRST_NAME_first_name', 'LAST_NAME': 'SALES_STAFF_LAST_NAME_last_name', 'ADDRESS1': 'SALES_STAFF_SALES_BRANCH_ADDRESS_address', 'ADDRESS2': 'SALES_STAFF_SALES_BRANCH_ADDRESS_EXTRA_address_extra', 'SALES_BRANCH_CODE' : 'SALES_STAFF_SALES_BRANCH_CODE_branch_code'})

all_dataframes.append(sales_staff)

sales_staff.head()

,SALES_STAFF_code,SALES_STAFF_email,SALES_STAFF_extension,SALES_STAFF_POSITION_EN_position,SALES_STAFF_WORK_PHONE_work_phone,SALES_STAFF_DATE_HIRED_hired,SALES_STAFF_MANAGER_CODE_manager,SALES_STAFF_FAX_fax,SALES_STAFF_FIRST_NAME_first_name,SALES_STAFF_LAST_NAME_last_name,SALES_STAFF_SALES_BRANCH_ADDRESS_address,SALES_STAFF_SALES_BRANCH_ADDRESS_EXTRA_address_extra,SALES_STAFF_SALES_BRANCH_CODE_branch_code
0,100,TSavolainen@grtd123.com,825,Level 2 Sales Representative,+358(0)17 - 433 127,23-Jul-1998 12:00:00 AM,18,+358(0)17 - 433 129,Tuomas,Savolainen,Kauppakatu 33,None,31
1,101,CKim@grtd123.com,1228,Level 2 Sales Representative,82-2-778-6587,15-Dec-1998 12:00:00 AM,87,82-2-778-6586,Chang-ho,Kim,"234-12, Kongdeok-Dong",Mapo-Ku,32
2,102,JChoi@grtd123.com,2641,Level 3 Sales Representative,82-2-778-6587,27-Nov-1998 12:00:00 AM,87,82-2-778-6586,Jung-ho,Choi,"234-12, Kongdeok-Dong",Mapo-Ku,32
3,103,KKwao@grtd123.com,1541,Level 2 Sales Representative,886-2-2975-1988,07-Oct-1998 12:00:00 AM,87,886-2-2989-4978,Kwei-Ping,Kao,"154, Choung-Cheng South Road",None,33
4,104,CFon@grtd123.com,2311,Level 3 Sales Representative,886-2-2975-1988,08-May-1996 12:00:00 AM,87,886-2-2989-4978,Chin-Tsai,Fang,"154, Choung-Cheng South Road",None,33


### Satisfaction_type

In [14]:
satisfaction_type = go_staff_satisfaction_type

# rename the columns
satisfaction_type = satisfaction_type.rename(columns={'SATISFACTION_TYPE_CODE': 'SATISFACTION_TYPE_code', 'DESCRIPTION': 'SATISFACTION_TYPE_description'})

all_dataframes.append(satisfaction_type)

satisfaction_type.head()

,SATISFACTION_TYPE_code,SATISFACTION_TYPE_DESCRIPTION,TRIAL633
0,1,Not satisfied,T
1,2,Less than satisfied,T
2,3,Satisfied,T
3,4,Very Satisfied,T
4,5,More than satisfied,T


### Course

In [15]:
course = go_staff_course

# rename the columns
course = course.rename(columns={'COURSE_CODE': 'COURSE_code', 'DESCRIPTION': 'COURSE_description'})

all_dataframes.append(course)

course.head()

,COURSE_code,COURSE_DESCRIPTION,TRIAL633
0,1,GO Orientation,T
1,2,GO Communication,T
2,3,GO Sales 1,T
3,4,GO Sales 2,T
4,5,GO Marketing 1,T


### Year

In [16]:
# to be used in training and satisfaction
# making year dataframe with year as the primary key
year = pd.DataFrame({'YEAR': pd.date_range(start='1/1/1990', end='1/1/2200').year})

# adding all the years from the satisfaction and training data.
year = pd.concat([year, pd.DataFrame({'YEAR': go_staff_training['YEAR']})])
year = pd.concat([year, pd.DataFrame({'YEAR': go_staff_satisfaction['YEAR']})])

# Reset the index
year = year.reset_index()

# Drop duplicates based on 'YEAR' column
year = year.drop_duplicates(subset='YEAR')

# Set 'YEAR' as the index again
year = year.set_index('YEAR')

all_dataframes.append(year)

year.head()

,index
YEAR,
1990,0
1991,365
1992,730
1993,1096
1994,1461


### Date

In [17]:
# to be used in order header
date = pd.DataFrame({'DATE_date': pd.date_range(start='1/1/1990', end='1/1/2200')})

# adding all the years from the order header data
date = pd.concat([date, pd.DataFrame({'DATE_date': go_sales_order_header['ORDER_DATE']})])

# setting the index
date['DATE_date'] = pd.to_datetime(date['DATE_date']) # convert to datetime

date = date.reset_index()

# drop duplicates
date = date.drop_duplicates(subset="DATE_date")

date = date.set_index('DATE_date')

all_dataframes.append(date)

date.head()

,index
DATE_date,
1990-01-01,0
1990-01-02,1
1990-01-03,2
1990-01-04,3
1990-01-05,4


### Order

In [18]:
# to be used in order header
# making order dataframe with order_number as the primary key 
order = pd.DataFrame({'ORDER_order_number': range(1, len(go_sales_order_method) + 1)})
order['ORDER_ORDER_METHOD_CODE_method_code'] = go_sales_order_method['ORDER_METHOD_CODE']
order['ORDER_ORDER_METHOD_EN_method'] = go_sales_order_method['ORDER_METHOD_EN']

order = order.set_index('ORDER_order_number')

all_dataframes.append(order)

order.head()

,ORDER_ORDER_METHOD_CODE_method_code,ORDER_ORDER_METHOD_EN_method
ORDER_order_number,,
1,1,Fax
2,2,Telephone
3,3,Mail
4,4,E-mail
5,5,Web


### Retailer_site

In [19]:
retailer_site = go_sales_retailer_site

# rename the columns
retailer_site = retailer_site.rename(columns={'RETAILER_SITE_CODE': 'RETAILER_SITE_code', 'COUNTRY_CODE': 'RETAILER_SITE_COUNTRY_CODE_country', 'CITY': 'RETAILER_SITE_CITY_city', 'REGION': 'RETAILER_SITE_REGION_region', 'POSTAL_ZONE': 'RETAILER_SITE_POSTAL_ZONE_postal_zone', 'RETAILER_CODE': 'RETAILER_SITE_RETAILER_CODE_retailer_code', 'ACTIVE_INDICATOR': 'RETAILER_SITE_ACTIVE_INDICATOR_indicator', 'ADDRESS1': 'RETAILER_SITE_ADDRESS1_address', 'ADDRESS2': 'RETAILER_SITE_ADDRESS2_address'})

all_dataframes.append(retailer_site)

retailer_site.head()

,RETAILER_SITE_code,RETAILER_SITE_RETAILER_CODE_retailer_code,RETAILER_SITE_ADDRESS1_address,RETAILER_SITE_ADDRESS2_address,RETAILER_SITE_CITY_city,RETAILER_SITE_REGION_region,RETAILER_SITE_POSTAL_ZONE_postal_zone,RETAILER_SITE_COUNTRY_CODE_country,RETAILER_SITE_ACTIVE_INDICATOR_indicator,TRIAL888
0,1,89,1117 Franklin Blvd,None,Winnipeg,Manitoba,R2C 0M5,4,1,T
1,10,92,"Suite 1500, 802 Docklands Street",None,Vancouver,British Columbia,V6E 3Z3,4,1,T
2,100,95,4830 Princeton,Unit 2,Albuquerque,New Mexico,87186,3,1,T
3,101,95,6630 Burnet Road,None,Austin,Texas,78769,3,1,T
4,102,108,2189 Louisville Road,None,Independence,Kentucky,41051,3,0,T


### Sales_branch

In [20]:
sales_branch = go_sales_sales_branch

# rename the columns
sales_branch = sales_branch.rename(columns={'SALES_BRANCH_CODE': 'SALES_BRANCH_code', 'COUNTRY_CODE': 'SALES_BRANCH_COUNTRY_CODE_country', 'REGION': 'SALES_BRANCH_REGION_region', 'CITY': 'SALES_BRANCH_CITY_city', 'POSTAL_ZONE': 'SALES_BRANCH_POSTAL_ZONE_postal_zone', 'ADDRESS1': 'SALES_BRANCH_ADDRESS1_address', 'ADDRESS2': 'SALES_BRANCH_ADDRESS2_address', 'MAIN_ADDRESS': 'SALES_BRANCH_MAIN_ADDRESS_address'})

all_dataframes.append(sales_branch)

sales_branch.head()

,SALES_BRANCH_code,SALES_BRANCH_ADDRESS1_address,SALES_BRANCH_ADDRESS2_address,SALES_BRANCH_CITY_city,SALES_BRANCH_REGION_region,SALES_BRANCH_POSTAL_ZONE_postal_zone,SALES_BRANCH_COUNTRY_CODE_country,TRIAL888
0,13,Schwabentor 35,None,Hamburg,None,D-22529,2,T
1,14,Leopoldstraße 36,None,München,None,D-81241,2,T
2,15,Isafjordsgatan 30 C,None,Kista,None,S-164 40,10,T
3,17,"7800, 756 - 6th Avenue. S.W.",None,Calgary,Alberta,T2P 3Z0,4,T
4,18,789 Yonge Street,None,Toronto,Ontario,M2M 4K8,4,T


### Retailer_contact

In [21]:
retailer_contact = go_crm_retailer_contact

# rename the columns
retailer_contact = retailer_contact.rename(columns={'RETAILER_CONTACT_CODE': 'RETAILER_CONTACT_code', 'E_MAIL': 'RETAILER_CONTACT_email', 'RETAILER_SITE_CODE': 'RETAILER_CONTACT_RETAILER_SITE_CODE_site_code', 'JOB_POSITION_EN': 'RETAILER_CONTACT_JOB_POSITION_EN_position', 'EXTENSION': 'RETAILER_CONTACT_EXTENSION_extension', 'FAX': 'RETAILER_CONTACT_FAX_fax', 'GENDER' : 'RETAILER_CONTACT_GENDER_gender', 'FIRST_NAME': 'RETAILER_CONTACT_FIRST_NAME_first_name', 'LAST_NAME': 'RETAILER_CONTACT_LAST_NAME_last_name'})

all_dataframes.append(retailer_contact)

retailer_contact.head()

,RETAILER_CONTACT_code,RETAILER_CONTACT_RETAILER_SITE_CODE_site_code,RETAILER_CONTACT_FIRST_NAME_first_name,RETAILER_CONTACT_LAST_NAME_last_name,RETAILER_CONTACT_JOB_POSITION_EN_position,RETAILER_CONTACT_EXTENSION_extension,RETAILER_CONTACT_FAX_fax,RETAILER_CONTACT_email,RETAILER_CONTACT_GENDER_gender,TRIAL222
0,10,14,Frank,Smith,Chief Purchaser,2489,1 (514) 741-1610,FSmith@legolfeurinc.com,M,T
1,100,85,Amanda,Yates,Stock Manager,None,1 (702) 369-2274,Y2883@emertxe.com,F,T
2,101,84,Paula,Schmidt,Assistant Purchaser,None,1 (702) 882-0978,S8832@emertxe.com,F,T
3,102,88,Micheal,Tao,District Manager,445,1 (713) 524-3215,T8839@emertxe.com,M,T
4,103,92,Jack,Murphy,District Manager,None,1 (305) 557-5478,Murphy@eyedimensions3.com,M,T


### Retailer

In [22]:
retailer = pd.merge(go_crm_retailer, go_crm_retailer_type, left_on='RETAILER_TYPE_CODE', right_on='RETAILER_TYPE_CODE')

# rename the columns
retailer = retailer.rename(columns={'RETAILER_CODE': 'RETAILER_code', 'COMPANY_NAME': 'RETAILER_name', 'RETAILER_CODEMR': 'RETAILER_COMPANY_CODE_MR_company', 'RETAILER_TYPE_CODE': 'RETAILER_RETAILER_TYPE_retailer_type_code', 'RETAILER_TYPE_EN': 'RETAILER_RETAILER_TYPE_retailer_type_EN'})

all_dataframes.append(retailer)

retailer.head()

,RETAILER_code,RETAILER_COMPANY_CODE_MR_company,RETAILER_name,RETAILER_RETAILER_TYPE_retailer_type_code,TRIAL219,RETAILER_RETAILER_TYPE_retailer_type_EN,TRIAL222
0,1,None,ActiForme,5,T,Equipment Rental Store,T
1,100,70,Golf Masters,1,T,Golf Shop,T
2,101,71,The Marketplace,2,T,Department Store,T
3,102,72,Camping Equipment Online,3,T,Direct Marketing,T
4,103,73,Tamarack Outfitter Rentals,5,T,Equipment Rental Store,T


### Product

In [23]:
product = pd.merge(go_sales_product, go_sales_product_type, left_on='PRODUCT_TYPE_CODE', right_on='PRODUCT_TYPE_CODE')
product = pd.merge(product, go_sales_product_line, left_on='PRODUCT_LINE_CODE', right_on='PRODUCT_LINE_CODE')

# rename the columns
product = product.rename(columns={'PRODUCT_NUMBER': 'PRODUCT_number', 'PRODUCT_NAME': 'PRODUCT_name_product', 'DESCRIPTION': 'PRODUCT_description_description', 'PRODUCT_IMAGE': 'PRODUCT_image_image', 'INTRODUCTION_DATE': 'PRODUCT_INTRODUCTION_DATE_introduced', 'PRODUCTION_COST': 'PRODUCT_PRODUCTION_COST_cost', 'MARGIN': 'PRODUCT_MARGIN_margin', 'LANGUAGE': 'PRODUCT_LANGUAGE_language',  'PRODUCT_TYPE_CODE': 'PRODUCT_PRODUCT_TYPE_code', 'PRODUCT_TYPE_EN': 'PRODUCT_PRODUCT_TYPE_code_en', 'PRODUCT_LINE_CODE': 'PRODUCT_PRODUCT_LINE_code', 'PRODUCT_LINE_EN': 'PRODUCT_PRODUCT_LINE_code_en'})

all_dataframes.append(product)

product.head()

,PRODUCT_number,PRODUCT_INTRODUCTION_DATE_introduced,PRODUCT_PRODUCT_TYPE_code,PRODUCT_PRODUCTION_COST_cost,PRODUCT_MARGIN_margin,PRODUCT_image_image,PRODUCT_LANGUAGE_language,PRODUCT_name_product,PRODUCT_description_description,TRIAL888_x,PRODUCT_PRODUCT_LINE_code,PRODUCT_PRODUCT_TYPE_code_en,TRIAL888_y,PRODUCT_PRODUCT_LINE_code_en,TRIAL888
0,1,15-2-2011,1,4,.33,P01CE1CG1.jpg,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",T,1,Cooking Gear,T,Camping Equipment,T
1,10,15-2-2011,1,10,.4,P10CE1CG1.jpg,EN,TrailChef Utensils,"Spoon, fork and knife set made of a light yet ...",T,1,Cooking Gear,T,Camping Equipment,T
2,100,15-2-2011,17,3,.5,P100OP4FA17.jpg,EN,Insect Bite Relief,The Insect Bite Relief helps the itching and s...,T,4,First Aid,T,Outdoor Protection,T
3,101,15-12-2019,18,305.54,.43,P101GE5IR18.jpg,EN,Hailstorm Steel Irons,Iron is 17-4 stainless steel. Shafts are grap...,T,5,Irons,T,Golf Equipment,T
4,102,10-12-2019,18,380.95,.51,P102GE5IR18.jpg,EN,Hailstorm Titanium Irons,Made entirely of pure titanium. The ultimate i...,T,5,Irons,T,Golf Equipment,T


### Order_details

In [24]:
order_details = go_sales_order_details

# replacing original columns with unit and returned_item
order_details['ORDER_DETAILS_UNIT_ID_unit'] = unit.index

# merging the returned_item and order_details
order_details = pd.merge(order_details, returned_item, left_on='ORDER_DETAIL_CODE', right_on='RETURNED_ITEM_ORDER_DETAIL_CODE_detail_code')

# merging unit and order_details
order_details = pd.merge(order_details, unit, left_on='UNIT_id', right_on='UNIT_id')

# dropping the columns replaced by unit and returned_item
order_details = order_details.drop(columns=['UNIT_COST', 'UNIT_PRICE', 'UNIT_SALE_PRICE']) 

# rename the columns
order_details = order_details.rename(columns={'ORDER_DETAILS_CODE': 'ORDER_DETAILS_code', 'QUANTITY': 'ORDER_DETAILS_QUANTITY_quantity', 'RETURNED_ITEM_code': 'ORDER_DETAILS_RETURN_CODE_returned', 'ORDER_NUMBER': 'ORDER_DETAILS_ORDER_NUMBER_order', 'PRODUCT_NUMBER': 'ORDER_DETAILS_PRODUCT_NUMBER_product', 'UNIT_ID': 'ORDER_DETAILS_UNIT_ID_unit'})

all_dataframes.append(order_details)

order_details.head()

,ORDER_DETAIL_CODE,ORDER_DETAILS_ORDER_NUMBER_order,ORDER_DETAILS_PRODUCT_NUMBER_product,ORDER_DETAILS_QUANTITY_quantity,TRIAL879,UNIT_id,ORDER_DETAILS_UNIT_ID_unit,ORDER_DETAILS_RETURN_CODE_returned,RETURNED_ITEM_DATE_date,RETURNED_ITEM_QUANTITY_quantity,RETURNED_ITEM_ORDER_DETAIL_CODE_detail_code,RETURNED_ITEM_RETURN_REASON_code,RETURNED_ITEM_RETURN_REASON_description_en,UNIT_COST_cost,UNIT_PRICE_price,UNIT_SALE_sale
0,100146,8465,15,16,T,144,143,1975,9-8-2021 18:45:17,16,100146,4,Wrong product shipped,504.54,756.82,593.5
1,100149,9116,16,64,T,147,146,1976,6-7-2021 13:34:59,64,100149,5,Unsatisfactory product,1.02,2.05,2.05
2,100199,9113,30,22,T,197,196,1979,6-7-2021 07:19:36,22,100199,4,Wrong product shipped,6.78,13.57,13.57
3,100250,9254,43,52,T,248,247,2298,14-12-2021 11:28:50,6,100250,1,Defective product,126.51,189.77,189.77
4,100641,9269,111,6,T,635,634,2301,19-12-2021 04:10:53,6,100641,4,Wrong product shipped,81.87,180.13,180.13


### Training

In [25]:
training = go_staff_training

# replacing the original columns with year and course
# merging the year and course
training = pd.merge(training, year, left_on='YEAR', right_on='YEAR')
training = pd.merge(training, course, left_on='COURSE_CODE', right_on='COURSE_code')

# rename the columns
training = training.rename(columns={'SALES_STAFF_CODE': 'TRAINING_SALES_STAFF_CODE', 'COURSE_CODE': 'TRAINING_COURSE_CODE', 'YEAR': 'TRAINING_YEAR'})

all_dataframes.append(training)

training.head()

,TRAINING_YEAR,TRAINING_SALES_STAFF_CODE,TRAINING_COURSE_CODE,TRIAL633_x,index,COURSE_code,COURSE_DESCRIPTION,TRIAL633_y
0,2004,100,2,T,0,2,GO Communication,T
1,2004,100,3,T,0,3,GO Sales 1,T
2,2004,100,4,T,0,4,GO Sales 2,T
3,2005,100,5,T,3,5,GO Marketing 1,T
4,2006,100,6,T,4,6,GO Marketing 2,T


### Satisfaction

In [26]:
satisfaction = go_staff_satisfaction

# replacing the original columns with year and satisfaction_type
# merging the year and satisfaction_type
satisfaction = pd.merge(satisfaction, year, left_on='YEAR', right_on='YEAR')
satisfaction = pd.merge(satisfaction, satisfaction_type, left_on='SATISFACTION_TYPE_CODE', right_on='SATISFACTION_TYPE_code')

# rename the columns
satisfaction = satisfaction.rename(columns={'SALES_STAFF_CODE': 'SATISFACTION_SALES_STAFF_CODE', 'SATISFACTION_TYPE_CODE': 'SATISFACTION_SATISFACTION_TYPE_CODE', 'YEAR': 'SATISFACTION_YEAR'})

all_dataframes.append(satisfaction)

satisfaction.head()

,SATISFACTION_YEAR,SATISFACTION_SALES_STAFF_CODE,SATISFACTION_SATISFACTION_TYPE_CODE,TRIAL633_x,index,SATISFACTION_TYPE_code,SATISFACTION_TYPE_DESCRIPTION,TRIAL633_y
0,2004,100,1,T,0,1,Not satisfied,T
1,2004,101,4,T,0,4,Very Satisfied,T
2,2004,102,4,T,0,4,Very Satisfied,T
3,2004,103,3,T,0,3,Satisfied,T
4,2004,104,4,T,0,4,Very Satisfied,T


### Order_header

In [27]:
order_header = pd.merge(go_sales_order_header, retailer, left_on='RETAILER_NAME', right_on='RETAILER_name')

# converting the date to datetime
order_header['ORDER_DATE'] = pd.to_datetime(order_header['ORDER_DATE'])

# replacing the original columns with date, sales_staff, retailer_site and retailer_contact
# merging the date, sales_staff, retailer_site and retailer_contact
order_header = pd.merge(order_header, date, left_on='ORDER_DATE', right_on='DATE_date')
order_header = pd.merge(order_header, sales_staff, left_on='SALES_STAFF_CODE', right_on='SALES_STAFF_code')
order_header = pd.merge(order_header, retailer_site, left_on='RETAILER_SITE_CODE', right_on='RETAILER_SITE_code')
order_header = pd.merge(order_header, retailer_contact, left_on='RETAILER_CONTACT_CODE', right_on='RETAILER_CONTACT_code')

# rename the columns
order_header = order_header.rename(columns={'ORDER_NUMBER': 'ORDER_HEADER_number', 'RETAILER_NAME': 'ORDER_HEADER_RETAILER_NAME', 'SALES_STAFF_CODE': 'ORDER_HEADER_SALES_STAFF', 'SALES_BRANCH_CODE': 'ORDER_HEADER_SALES_BRANCH_CODE', 'ORDER_DATE': 'ORDER_HEADER_ORDER_DATE', 'RETAILER_SITE_CODE': 'ORDER_HEADER_RETAILER_SITE_CODE', 'RETAILER_CONTACT_CODE': 'ORDER_HEADER_RETAILER_CONTACT_CODE'})

all_dataframes.append(order_header)

order_header.head()

,ORDER_HEADER_number,ORDER_HEADER_RETAILER_NAME,ORDER_HEADER_RETAILER_SITE_CODE,ORDER_HEADER_RETAILER_CONTACT_CODE,ORDER_HEADER_SALES_STAFF,ORDER_HEADER_SALES_BRANCH_CODE,ORDER_HEADER_ORDER_DATE,ORDER_METHOD_CODE,TRIAL885,RETAILER_code,...,RETAILER_CONTACT_code,RETAILER_CONTACT_RETAILER_SITE_CODE_site_code,RETAILER_CONTACT_FIRST_NAME_first_name,RETAILER_CONTACT_LAST_NAME_last_name,RETAILER_CONTACT_JOB_POSITION_EN_position,RETAILER_CONTACT_EXTENSION_extension,RETAILER_CONTACT_FAX_fax,RETAILER_CONTACT_email,RETAILER_CONTACT_GENDER_gender,TRIAL222_y
0,1153,Ultra Sports,4,6,50,18,2020-04-14,7,T,90,...,6,4,Marie,Turcotte,Assistant Purchaser,635,1 (514) 747-2582,MTurcotte@cybserv19.com,F,T
1,1154,Ultra Sports,4,6,50,18,2020-10-01,4,T,90,...,6,4,Marie,Turcotte,Assistant Purchaser,635,1 (514) 747-2582,MTurcotte@cybserv19.com,F,T
2,1155,Ultra Sports,5,65,49,18,2020-04-21,7,T,90,...,65,5,Andrea,Martens,Stock Manager,None,1 (705) 675-9848,AMartens@cybserv19.com,F,T
3,1156,Ultra Sports,5,65,49,18,2020-09-09,5,T,90,...,65,5,Andrea,Martens,Stock Manager,None,1 (705) 675-9848,AMartens@cybserv19.com,F,T
4,1157,Ultra Sports,6,66,50,18,2020-04-10,2,T,90,...,66,6,Bill,Follett,Site Assistant Manager,None,1 (819) 427-4397,BFollett@cybserv19.com,M,T


### GO_SALES_INVENTORY_LEVELS

In [28]:
# adding a leading 0 to the month if it is a single digit
go_sales_inventory_levels['INVENTORY_MONTH'] = go_sales_inventory_levels['INVENTORY_MONTH'].astype(str).str.zfill(2)

# combine year and month to one column
go_sales_inventory_levels['YEAR_MONTH'] = go_sales_inventory_levels['INVENTORY_YEAR'].astype(str) + '-' + go_sales_inventory_levels['INVENTORY_MONTH'].astype(str)

# dropping the original columns
go_sales_inventory_levels = go_sales_inventory_levels.drop(columns=['INVENTORY_YEAR', 'INVENTORY_MONTH'])

go_sales_inventory_levels = go_sales_inventory_levels.rename(columns={'INVENTORY_COUNT': 'GO_SALES_INVENTORY_LEVELS_INVENTORY_COUNT', 'PRODUCT_NUMBER': 'GO_SALES_INVENTORY_LEVELS_PRODUCT_NUMBER', 'YEAR_MONTH': 'GO_SALES_INVENTORY_LEVELS_YEAR_MONTH'})

all_dataframes.append(go_sales_inventory_levels)

go_sales_inventory_levels.head()

,GO_SALES_INVENTORY_LEVELS_PRODUCT_NUMBER,GO_SALES_INVENTORY_LEVELS_INVENTORY_COUNT,GO_SALES_INVENTORY_LEVELS_YEAR_MONTH
0,48,1932,2021-04
1,49,1400,2021-04
2,50,21705,2021-04
3,51,9710,2021-04
4,52,5616,2021-04


### GO_SALES_PRODUCT_FORECAST

In [29]:
# adding a leading zero to the month if it is a single digit
go_sales_product_forcast['MONTH'] = go_sales_product_forcast['MONTH'].astype(str).str.zfill(2)

#combine year and month to one column
go_sales_product_forcast['YEAR_MONTH'] = go_sales_product_forcast['YEAR'].astype(str) + '-' + go_sales_product_forcast['MONTH'].astype(str)

# dropping the original columns
go_sales_product_forcast = go_sales_product_forcast.drop(columns=['YEAR', 'MONTH'])

go_sales_product_forecast = go_sales_product_forcast.rename(columns={'EXPECTED_VOLUME': 'GO_SALES_PRODUCT_FORECAST_EXPECTED_VOLUME', 'EXPECTED_COST': 'GO_SALES_PRODUCT_FORECAST_EXPECTED_COST', 'EXPECTED_MARGIN': 'GO_SALES_PRODUCT_FORECAST_EXPECTED_MARGIN', 'PRODUCT_NUMBER': 'GO_SALES_PRODUCT_FORECAST_PRODUCT_NUMBER', 'YEAR_MONTH': 'GO_SALES_PRODUCT_FORECAST_YEAR_MONTH'})

all_dataframes.append(go_sales_product_forecast)

go_sales_product_forecast.head()

,GO_SALES_PRODUCT_FORECAST_PRODUCT_NUMBER,GO_SALES_PRODUCT_FORECAST_EXPECTED_VOLUME,GO_SALES_PRODUCT_FORECAST_YEAR_MONTH
0,44,383,2022-12
1,45,80,2021-01
2,45,51,2021-02
3,45,214,2021-03
4,45,300,2021-04


###  SALES_TARGETData

In [30]:
# merge product, retailer and sales_staff
go_sales_sales_target_data = pd.merge(go_sales_sales_target_data, product, left_on='PRODUCT_NUMBER', right_on='PRODUCT_number', suffixes=('_product', '_retailer'))

go_sales_sales_target_data = pd.merge(go_sales_sales_target_data, retailer, left_on='RETAILER_CODE', right_on='RETAILER_code', suffixes=('_retailer', '_sales_staff'))

# delete the original columns that are replaced by product, retailer and sales_staff
go_sales_sales_target_data = go_sales_sales_target_data.drop(columns=['PRODUCT_NUMBER', 'RETAILER_CODE', 'RETAILER_name', 'RETAILER_COMPANY_CODE_MR_company', 'RETAILER_RETAILER_TYPE_retailer_type_code', 'RETAILER_RETAILER_TYPE_retailer_type_EN'])

sales_target_data = go_sales_sales_target_data.rename(columns={'SALES_YEAR': 'SALES_TARGETDATA_SALES_YEAR', 'SALES_PERIOD': 'SALES_TARGETDATA_SALES_PERIOD', 'RETAILER_NAME': 'SALES_TARGETDATA_RETAILER_NAME', 'SALES_TARGET': 'SALES_TARGETDATA_SALES_TARGET', 'TARGET_COST': 'SALES_TARGETDATA_TARGET_COST', 'TARGET_MARGIN': 'SALES_TARGETDATA_TARGET_MARGIN', 'SALES_STAFF_CODE': 'SALES_TARGETDATA_SALES_STAFF_CODE', 'PRODUCT_NUMBER': 'SALES_TARGETDATA_PRODUCT_NUMBER', 'RETAILER_CODE': 'SALES_TARGETDATA_RETAILER_CODE'})

all_dataframes.append(sales_target_data)

sales_target_data.head()

,Id,SALES_TARGETDATA_SALES_STAFF_CODE,SALES_TARGETDATA_SALES_YEAR,SALES_TARGETDATA_SALES_PERIOD,SALES_TARGETDATA_RETAILER_NAME,SALES_TARGETDATA_SALES_TARGET,TRIAL888_product,PRODUCT_number,PRODUCT_INTRODUCTION_DATE_introduced,PRODUCT_PRODUCT_TYPE_code,...,PRODUCT_description_description,TRIAL888_x,PRODUCT_PRODUCT_LINE_code,PRODUCT_PRODUCT_TYPE_code_en,TRIAL888_y,PRODUCT_PRODUCT_LINE_code_en,TRIAL888_retailer,RETAILER_code,TRIAL219,TRIAL222
0,1,6,2021,5,Altitudes extrêmes,489,T,53,10-1-2021,8,...,This climbing lamp is water-resistant. It come...,T,2,Climbing Accessories,T,Mountaineering Equipment,T,33,T,T
1,2,6,2021,5,Altitudes extrêmes,666,T,55,10-1-2021,8,...,Rechargeable battery pack for the Firefly Clim...,T,2,Climbing Accessories,T,Mountaineering Equipment,T,33,T,T
2,3,6,2021,5,Altitudes extrêmes,682,T,56,10-1-2021,8,...,Nylon web bag perfect for holding several stic...,T,2,Climbing Accessories,T,Mountaineering Equipment,T,33,T,T
3,4,6,2021,5,Camping Sauvage,3193,T,76,15-2-2011,12,...,Knife is 33 cm long with a 20 cm blade made of...,T,3,Knives,T,Personal Accessories,T,35,T,T
4,5,6,2021,5,Camping Sauvage,7236,T,85,12-6-2015,14,...,Hand held GPS receiver with color display. Inc...,T,3,Navigation,T,Personal Accessories,T,35,T,T


### Retailer_segment

In [31]:
retailer_segment = go_crm_retailer_segment

# rename the columns
retailer_segment = retailer_segment.rename(columns={'SEGMENT_CODE': 'RETAILER_SEGMENT_segment_code', 'LANGUAGE': 'RETAILER_SEGMENT_language', 'SEGMENT_NAME': 'RETAILER_SEGMENT_segment_name', 'SEGMENT_DESCRIPTION': 'RETAILER_SEGMENT_SEGMENT_DESCRIPTION_description'})

all_dataframes.append(retailer_segment)

retailer_segment.head()

,RETAILER_SEGMENT_segment_code,RETAILER_SEGMENT_language,RETAILER_SEGMENT_segment_name,RETAILER_SEGMENT_SEGMENT_DESCRIPTION_description,TRIAL222
0,1,EN,Outdoors Shop - Exclusive,"Primarily offers camping supplies, hiking equi...",T
1,10,EN,Eyewear Store,Primarily offers prescription and fashion eyew...,T
2,11,EN,Sports Store - Exclusive,Primarily offers a wide range of sporting good...,T
3,12,EN,Sports Store,Primarily offers a wide range of sporting goods.,T
4,2,EN,Outdoors Shop,"Primarily offers camping supplies, hiking equi...",T


### Retailer_headquarters

In [32]:
retailer_headquarters = go_crm_retailer_headquarters

# rename the columns
retailer_headquarters = retailer_headquarters.rename(columns={'RETAILER_CODEMR': 'RETAILER_HEADQUARTER_codemr', 'RETAILER_NAME' : 'RETAIL_HEADQUARTER_retailer_name', 'ADDRESS1' : 'RETAILER_HEADQUARTER_address1_address', 'ADDRESS2' : 'RETAILER_HEADQUARTER_address2_address', 'CITY' : 'RETAILER_HEADQUARTER_city_city', 'REGION' : 'RETAILER_HEADQUARTER_region_region', 'POSTAL_ZONE' : 'RETAILER_HEADQUARTER_postal_zone_postal_zone', 'COUNTRY_CODE' : 'RETAILER_HEADQUARTER_country_code_country', 'PHONE' : 'RETAILER_HEADQUARTER_phone_phone', 'FAX': 'RETAILER_HEADQUARTER_fax_fax', 'SEGMENT_CODE' : 'RETAILER_HEADQUARTER_segment_code'})

all_dataframes.append(retailer_headquarters)

retailer_headquarters.head()

,RETAILER_HEADQUARTER_codemr,RETAIL_HEADQUARTER_retailer_name,RETAILER_HEADQUARTER_address1_address,RETAILER_HEADQUARTER_address2_address,RETAILER_HEADQUARTER_city_city,RETAILER_HEADQUARTER_region_region,RETAILER_HEADQUARTER_postal_zone_postal_zone,RETAILER_HEADQUARTER_country_code_country,RETAILER_HEADQUARTER_phone_phone,RETAILER_HEADQUARTER_fax_fax,RETAILER_HEADQUARTER_segment_code,TRIAL222
0,1,Rock Steady,"45, rue Atwater",None,Montréal,Québec,H2T 9K8,4,1 (514) 747-4481,1 (514) 741-2277,2,T
1,10,Excellence en Montagne,"160, rue Moureaux",Bureau 410,Ville de Québec,Québec,G1B 3L5,4,1 (418) 653-6456,1 (418) 653-2311,2,T
2,11,Vacation Central,"3000, 58 Merivale Road",None,Ottawa,Ontario,K0T 9R0,4,1 (613) 736-6443,1 (613) 736-5593,6,T
3,12,Outdoor Central Ltd,"Suite 23, 180 McKercher Drive",None,Regina,Saskatchewan,S6J 3G0,4,1 (306) 243-8764,1 (306) 243-5213,2,T
4,13,Precipice Equipment,23 Blackcomb Way,None,Whistler,British Columbia,V6T 1Y8,4,1 (604) 938-7564,1 (604) 938-2324,1,T


### Verwijderen trial kolommen

In [33]:
for df in all_dataframes:
    trial_columns = df.filter(like='TRIAL').columns
    df.drop(columns=trial_columns, inplace=True)

# Omzetten van de data
Omzetten van de data naar de juiste data types.
Year en Date zijn al omgezet naar de juiste data types.
GO_SALES_INVENTORY_LEVELS en GO_SALES_PRODUCT_FORECAST hebben ook al de juiste datatypen

### Unit

In [34]:
unit['UNIT_COST_cost'] = unit['UNIT_COST_cost'].astype('float64')
unit['UNIT_PRICE_price'] = unit['UNIT_PRICE_price'].astype('float64')
unit['UNIT_SALE_sale'] = unit['UNIT_SALE_sale'].astype('float64')

unit.dtypes

UNIT_COST_cost      float64
UNIT_PRICE_price    float64
UNIT_SALE_sale      float64
UNIT_id               int64
dtype: object

### Sales_staff

In [35]:
sales_staff['SALES_STAFF_code'] = sales_staff['SALES_STAFF_code'].astype('int')
sales_staff['SALES_STAFF_email'] = sales_staff['SALES_STAFF_email'].astype('string')
sales_staff['SALES_STAFF_extension'] = sales_staff['SALES_STAFF_extension'].astype('string')
sales_staff['SALES_STAFF_POSITION_EN_position'] = sales_staff['SALES_STAFF_POSITION_EN_position'].astype('string')
sales_staff['SALES_STAFF_WORK_PHONE_work_phone'] = sales_staff['SALES_STAFF_WORK_PHONE_work_phone'].astype('string')
sales_staff['SALES_STAFF_DATE_HIRED_hired'] = pd.to_datetime(sales_staff['SALES_STAFF_DATE_HIRED_hired'], format='mixed')
sales_staff['SALES_STAFF_MANAGER_CODE_manager'] = sales_staff['SALES_STAFF_MANAGER_CODE_manager'].astype('int64')
sales_staff['SALES_STAFF_FAX_fax'] = sales_staff['SALES_STAFF_FAX_fax'].astype('string')
sales_staff['SALES_STAFF_FIRST_NAME_first_name'] = sales_staff['SALES_STAFF_FIRST_NAME_first_name'].astype('string')
sales_staff['SALES_STAFF_LAST_NAME_last_name'] = sales_staff['SALES_STAFF_LAST_NAME_last_name'].astype('string')
sales_staff['SALES_STAFF_SALES_BRANCH_CODE_branch_code'] = sales_staff['SALES_STAFF_SALES_BRANCH_CODE_branch_code'].astype('int64')
sales_staff['SALES_STAFF_SALES_BRANCH_ADDRESS_address'] = sales_staff['SALES_STAFF_SALES_BRANCH_ADDRESS_address'].astype('string')
sales_staff['SALES_STAFF_SALES_BRANCH_ADDRESS_EXTRA_address_extra'] = sales_staff['SALES_STAFF_SALES_BRANCH_ADDRESS_EXTRA_address_extra'].astype('string')

sales_staff.dtypes

SALES_STAFF_code                                                 int32
SALES_STAFF_email                                       string[python]
SALES_STAFF_extension                                   string[python]
SALES_STAFF_POSITION_EN_position                        string[python]
SALES_STAFF_WORK_PHONE_work_phone                       string[python]
SALES_STAFF_DATE_HIRED_hired                            datetime64[ns]
SALES_STAFF_MANAGER_CODE_manager                                 int64
SALES_STAFF_FAX_fax                                     string[python]
SALES_STAFF_FIRST_NAME_first_name                       string[python]
SALES_STAFF_LAST_NAME_last_name                         string[python]
SALES_STAFF_SALES_BRANCH_ADDRESS_address                string[python]
SALES_STAFF_SALES_BRANCH_ADDRESS_EXTRA_address_extra    string[python]
SALES_STAFF_SALES_BRANCH_CODE_branch_code                        int64
dtype: object

### satisfaction_type

In [36]:
satisfaction_type['SATISFACTION_TYPE_code'] = satisfaction_type['SATISFACTION_TYPE_code'].astype('int')
satisfaction_type['SATISFACTION_TYPE_DESCRIPTION'] = satisfaction_type['SATISFACTION_TYPE_DESCRIPTION'].astype('string')

satisfaction_type.dtypes

SATISFACTION_TYPE_code                    int32
SATISFACTION_TYPE_DESCRIPTION    string[python]
dtype: object

### Course

In [37]:
course['COURSE_code'] = course['COURSE_code'].astype('int')
course['COURSE_DESCRIPTION'] = course['COURSE_DESCRIPTION'].astype('string')

course.dtypes

COURSE_code                    int32
COURSE_DESCRIPTION    string[python]
dtype: object

### Order

In [38]:
order['ORDER_ORDER_METHOD_CODE_method_code'] = order['ORDER_ORDER_METHOD_CODE_method_code'].astype('int')
order['ORDER_ORDER_METHOD_EN_method'] = order['ORDER_ORDER_METHOD_EN_method'].astype('string')

# generating the primary key
order['ORDER_order_number'] = range(1, len(order) + 1)

# setting the index
order = order.set_index('ORDER_order_number')

order.dtypes

ORDER_ORDER_METHOD_CODE_method_code             int32
ORDER_ORDER_METHOD_EN_method           string[python]
dtype: object

### Retailer_site

In [39]:
retailer_site['RETAILER_SITE_code'] = retailer_site['RETAILER_SITE_code'].astype('int')
retailer_site['RETAILER_SITE_COUNTRY_CODE_country'] = retailer_site['RETAILER_SITE_COUNTRY_CODE_country'].astype('string')
retailer_site['RETAILER_SITE_CITY_city'] = retailer_site['RETAILER_SITE_CITY_city'].astype('string')
retailer_site['RETAILER_SITE_REGION_region'] = retailer_site['RETAILER_SITE_REGION_region'].astype('string')
retailer_site['RETAILER_SITE_POSTAL_ZONE_postal_zone'] = retailer_site['RETAILER_SITE_POSTAL_ZONE_postal_zone'].astype('string')
retailer_site['RETAILER_SITE_RETAILER_CODE_retailer_code'] = retailer_site['RETAILER_SITE_RETAILER_CODE_retailer_code'].astype('int')
retailer_site['RETAILER_SITE_ACTIVE_INDICATOR_indicator'] = retailer_site['RETAILER_SITE_ACTIVE_INDICATOR_indicator'].astype('int')
retailer_site['RETAILER_SITE_ADDRESS1_address'] = retailer_site['RETAILER_SITE_ADDRESS1_address'].astype('string')
retailer_site['RETAILER_SITE_ADDRESS2_address'] = retailer_site['RETAILER_SITE_ADDRESS2_address'].astype('string')

retailer_site.dtypes

RETAILER_SITE_code                                    int32
RETAILER_SITE_RETAILER_CODE_retailer_code             int32
RETAILER_SITE_ADDRESS1_address               string[python]
RETAILER_SITE_ADDRESS2_address               string[python]
RETAILER_SITE_CITY_city                      string[python]
RETAILER_SITE_REGION_region                  string[python]
RETAILER_SITE_POSTAL_ZONE_postal_zone        string[python]
RETAILER_SITE_COUNTRY_CODE_country           string[python]
RETAILER_SITE_ACTIVE_INDICATOR_indicator              int32
dtype: object

### Sales_branch

In [40]:
sales_branch['SALES_BRANCH_code'] = sales_branch['SALES_BRANCH_code'].astype('int')
sales_branch['SALES_BRANCH_COUNTRY_CODE_country'] = sales_branch['SALES_BRANCH_COUNTRY_CODE_country'].astype('string')
sales_branch['SALES_BRANCH_REGION_region'] = sales_branch['SALES_BRANCH_REGION_region'].astype('string')
sales_branch['SALES_BRANCH_CITY_city'] = sales_branch['SALES_BRANCH_CITY_city'].astype('string')
sales_branch['SALES_BRANCH_POSTAL_ZONE_postal_zone'] = sales_branch['SALES_BRANCH_POSTAL_ZONE_postal_zone'].astype('string')
sales_branch['SALES_BRANCH_ADDRESS1_address'] = sales_branch['SALES_BRANCH_ADDRESS1_address'].astype('string')
sales_branch['SALES_BRANCH_ADDRESS2_address'] = sales_branch['SALES_BRANCH_ADDRESS2_address'].astype('string')

sales_branch.dtypes

SALES_BRANCH_code                                int32
SALES_BRANCH_ADDRESS1_address           string[python]
SALES_BRANCH_ADDRESS2_address           string[python]
SALES_BRANCH_CITY_city                  string[python]
SALES_BRANCH_REGION_region              string[python]
SALES_BRANCH_POSTAL_ZONE_postal_zone    string[python]
SALES_BRANCH_COUNTRY_CODE_country       string[python]
dtype: object

### Retailer_contact

In [41]:
retailer_contact['RETAILER_CONTACT_code'] = retailer_contact['RETAILER_CONTACT_code'].astype('int')
retailer_contact['RETAILER_CONTACT_email'] = retailer_contact['RETAILER_CONTACT_email'].astype('string')
retailer_contact['RETAILER_CONTACT_RETAILER_SITE_CODE_site_code'] = retailer_contact['RETAILER_CONTACT_RETAILER_SITE_CODE_site_code'].astype('int')
retailer_contact['RETAILER_CONTACT_JOB_POSITION_EN_position'] = retailer_contact['RETAILER_CONTACT_JOB_POSITION_EN_position'].astype('string')
retailer_contact['RETAILER_CONTACT_EXTENSION_extension'] = retailer_contact['RETAILER_CONTACT_EXTENSION_extension'].astype('string')
retailer_contact['RETAILER_CONTACT_FAX_fax'] = retailer_contact['RETAILER_CONTACT_FAX_fax'].astype('string')
retailer_contact['RETAILER_CONTACT_GENDER_gender'] = retailer_contact['RETAILER_CONTACT_GENDER_gender'].astype('string')
retailer_contact['RETAILER_CONTACT_FIRST_NAME_first_name'] = retailer_contact['RETAILER_CONTACT_FIRST_NAME_first_name'].astype('string')
retailer_contact['RETAILER_CONTACT_LAST_NAME_last_name'] = retailer_contact['RETAILER_CONTACT_LAST_NAME_last_name'].astype('string')

retailer_contact.dtypes

RETAILER_CONTACT_code                                     int32
RETAILER_CONTACT_RETAILER_SITE_CODE_site_code             int32
RETAILER_CONTACT_FIRST_NAME_first_name           string[python]
RETAILER_CONTACT_LAST_NAME_last_name             string[python]
RETAILER_CONTACT_JOB_POSITION_EN_position        string[python]
RETAILER_CONTACT_EXTENSION_extension             string[python]
RETAILER_CONTACT_FAX_fax                         string[python]
RETAILER_CONTACT_email                           string[python]
RETAILER_CONTACT_GENDER_gender                   string[python]
dtype: object

### Retailer

In [42]:
retailer['RETAILER_code'] = retailer['RETAILER_code'].astype('int')
retailer['RETAILER_name'] = retailer['RETAILER_name'].astype('string')
retailer['RETAILER_COMPANY_CODE_MR_company'] = retailer['RETAILER_COMPANY_CODE_MR_company'].astype('string')
retailer['RETAILER_RETAILER_TYPE_retailer_type_code'] = retailer['RETAILER_RETAILER_TYPE_retailer_type_code'].astype('int')
retailer['RETAILER_RETAILER_TYPE_retailer_type_EN'] = retailer['RETAILER_RETAILER_TYPE_retailer_type_EN'].astype('string')

retailer.dtypes

RETAILER_code                                         int32
RETAILER_COMPANY_CODE_MR_company             string[python]
RETAILER_name                                string[python]
RETAILER_RETAILER_TYPE_retailer_type_code             int32
RETAILER_RETAILER_TYPE_retailer_type_EN      string[python]
dtype: object

### Product

In [43]:
product['PRODUCT_number'] = product['PRODUCT_number'].astype('int')
product['PRODUCT_name_product'] = product['PRODUCT_name_product'].astype('string')
product['PRODUCT_description_description'] = product['PRODUCT_description_description'].astype('string')
product['PRODUCT_image_image'] = product['PRODUCT_image_image'].astype('string')
product['PRODUCT_INTRODUCTION_DATE_introduced'] = pd.to_datetime(product['PRODUCT_INTRODUCTION_DATE_introduced'], format='mixed')
product['PRODUCT_PRODUCTION_COST_cost'] = product['PRODUCT_PRODUCTION_COST_cost'].astype('float')
product['PRODUCT_MARGIN_margin'] = product['PRODUCT_MARGIN_margin'].astype('float')
product['PRODUCT_LANGUAGE_language'] = product['PRODUCT_LANGUAGE_language'].astype('string')
product['PRODUCT_PRODUCT_LINE_code'] = product['PRODUCT_PRODUCT_LINE_code'].astype('string')
product['PRODUCT_PRODUCT_LINE_code_en'] = product['PRODUCT_PRODUCT_LINE_code_en'].astype('string')
product['PRODUCT_PRODUCT_TYPE_code'] = product['PRODUCT_PRODUCT_TYPE_code'].astype('string')
product['PRODUCT_PRODUCT_TYPE_code_en'] = product['PRODUCT_PRODUCT_TYPE_code_en'].astype('string')

product.dtypes

PRODUCT_number                                   int32
PRODUCT_INTRODUCTION_DATE_introduced    datetime64[ns]
PRODUCT_PRODUCT_TYPE_code               string[python]
PRODUCT_PRODUCTION_COST_cost                   float64
PRODUCT_MARGIN_margin                          float64
PRODUCT_image_image                     string[python]
PRODUCT_LANGUAGE_language               string[python]
PRODUCT_name_product                    string[python]
PRODUCT_description_description         string[python]
PRODUCT_PRODUCT_LINE_code               string[python]
PRODUCT_PRODUCT_TYPE_code_en            string[python]
PRODUCT_PRODUCT_LINE_code_en            string[python]
dtype: object

### Order_details

In [44]:
order_details['ORDER_DETAILS_QUANTITY_quantity'] = order_details['ORDER_DETAILS_QUANTITY_quantity'].astype('int')
order_details['ORDER_DETAILS_RETURN_CODE_returned'] = order_details['ORDER_DETAILS_RETURN_CODE_returned'].astype('int')
order_details['ORDER_DETAILS_ORDER_NUMBER_order'] = order_details['ORDER_DETAILS_ORDER_NUMBER_order'].astype('int')
order_details['ORDER_DETAILS_PRODUCT_NUMBER_product'] = order_details['ORDER_DETAILS_PRODUCT_NUMBER_product'].astype('int')
order_details['RETURNED_ITEM_DATE_date'] = pd.to_datetime(order_details['RETURNED_ITEM_DATE_date'], format='mixed')
order_details['RETURNED_ITEM_QUANTITY_quantity'] = order_details['RETURNED_ITEM_QUANTITY_quantity'].astype('int')
order_details['RETURNED_ITEM_ORDER_DETAIL_CODE_detail_code'] = order_details['RETURNED_ITEM_ORDER_DETAIL_CODE_detail_code'].astype('int')
order_details['RETURNED_ITEM_RETURN_REASON_code'] = order_details['RETURNED_ITEM_RETURN_REASON_code'].astype('int')
order_details['RETURNED_ITEM_RETURN_REASON_description_en'] = order_details['RETURNED_ITEM_RETURN_REASON_description_en'].astype('string')
order_details['UNIT_COST_cost'] = order_details['UNIT_COST_cost'].astype('float64')
order_details['UNIT_PRICE_price'] = order_details['UNIT_PRICE_price'].astype('float64')
order_details['UNIT_SALE_sale'] = order_details['UNIT_SALE_sale'].astype('float64')

# unit is already converted
order_details.dtypes

ORDER_DETAIL_CODE                                      object
ORDER_DETAILS_ORDER_NUMBER_order                        int32
ORDER_DETAILS_PRODUCT_NUMBER_product                    int32
ORDER_DETAILS_QUANTITY_quantity                         int32
UNIT_id                                                 int64
ORDER_DETAILS_UNIT_ID_unit                              int64
ORDER_DETAILS_RETURN_CODE_returned                      int32
RETURNED_ITEM_DATE_date                        datetime64[ns]
RETURNED_ITEM_QUANTITY_quantity                         int32
RETURNED_ITEM_ORDER_DETAIL_CODE_detail_code             int32
RETURNED_ITEM_RETURN_REASON_code                        int32
RETURNED_ITEM_RETURN_REASON_description_en     string[python]
UNIT_COST_cost                                        float64
UNIT_PRICE_price                                      float64
UNIT_SALE_sale                                        float64
dtype: object

### Returned_item

In [45]:
returned_item['RETURNED_ITEM_code'] = returned_item['RETURNED_ITEM_code'].astype('int')
returned_item['RETURNED_ITEM_DATE'] = pd.to_datetime(returned_item['RETURNED_ITEM_DATE'], format='mixed')
returned_item['RETURNED_ITEM_QUANTITY_quantity'] = returned_item['RETURNED_ITEM_QUANTITY_quantity'].astype('int')
returned_item['RETURNED_ITEM_ORDER_DETAIL_CODE_detail_code'] = returned_item['RETURNED_ITEM_ORDER_DETAIL_CODE_detail_code'].astype('int')
returned_item['RETURNED_ITEM_RETURN_REASON_code'] = returned_item['RETURNED_ITEM_RETURN_REASON_code'].astype('int')
returned_item['RETURNED_ITEM_RETURN_REASON_description_en'] = returned_item['RETURNED_ITEM_RETURN_REASON_description_en'].astype('string')

returned_item.dtypes

RETURNED_ITEM_code                                      int32
RETURNED_ITEM_DATE_date                        datetime64[ns]
RETURNED_ITEM_QUANTITY_quantity                         int32
RETURNED_ITEM_ORDER_DETAIL_CODE_detail_code             int32
RETURNED_ITEM_RETURN_REASON_code                        int32
RETURNED_ITEM_RETURN_REASON_description_en     string[python]
dtype: object

### Sales_Targetdata

In [46]:
sales_target_data['SALES_TARGETDATA_SALES_YEAR'] = sales_target_data['SALES_TARGETDATA_SALES_YEAR'].astype('int')
sales_target_data['SALES_TARGETDATA_SALES_PERIOD'] = sales_target_data['SALES_TARGETDATA_SALES_PERIOD'].astype('string')
sales_target_data['SALES_TARGETDATA_RETAILER_NAME'] = sales_target_data['SALES_TARGETDATA_RETAILER_NAME'].astype('string')
sales_target_data['SALES_TARGETDATA_SALES_TARGET'] = sales_target_data['SALES_TARGETDATA_SALES_TARGET'].astype('float64')
sales_target_data['SALES_TARGETDATA_SALES_STAFF_CODE'] = sales_target_data['SALES_TARGETDATA_SALES_STAFF_CODE'].astype('int')
sales_target_data['PRODUCT_number'] = sales_target_data['PRODUCT_number'].astype('int')
sales_target_data['PRODUCT_INTRODUCTION_DATE_introduced'] = pd.to_datetime(sales_target_data['PRODUCT_INTRODUCTION_DATE_introduced'], format='mixed')
sales_target_data['PRODUCT_PRODUCTION_COST_cost'] = sales_target_data['PRODUCT_PRODUCTION_COST_cost'].astype('float64')
sales_target_data['PRODUCT_PRODUCT_TYPE_code'] = sales_target_data['PRODUCT_PRODUCT_TYPE_code'].astype('string')
sales_target_data['PRODUCT_PRODUCT_TYPE_code_en'] = sales_target_data['PRODUCT_PRODUCT_TYPE_code_en'].astype('string')
sales_target_data['PRODUCT_PRODUCT_LINE_code'] = sales_target_data['PRODUCT_PRODUCT_LINE_code'].astype('string')
sales_target_data['PRODUCT_PRODUCT_LINE_code_en'] = sales_target_data['PRODUCT_PRODUCT_LINE_code_en'].astype('string')
sales_target_data['RETAILER_code'] = sales_target_data['RETAILER_code'].astype('int')
sales_target_data['PRODUCT_MARGIN_margin'] = sales_target_data['PRODUCT_MARGIN_margin'].astype('float64')
sales_target_data['PRODUCT_image_image'] = sales_target_data['PRODUCT_image_image'].astype('string')
sales_target_data['PRODUCT_LANGUAGE_language'] = sales_target_data['PRODUCT_LANGUAGE_language'].astype('string')
sales_target_data['PRODUCT_name_product'] = sales_target_data['PRODUCT_name_product'].astype('string')
sales_target_data['PRODUCT_description_description'] = sales_target_data['PRODUCT_description_description'].astype('string')




sales_target_data.dtypes

Id                                               int64
SALES_TARGETDATA_SALES_STAFF_CODE                int32
SALES_TARGETDATA_SALES_YEAR                      int32
SALES_TARGETDATA_SALES_PERIOD           string[python]
SALES_TARGETDATA_RETAILER_NAME          string[python]
SALES_TARGETDATA_SALES_TARGET                  float64
PRODUCT_number                                   int32
PRODUCT_INTRODUCTION_DATE_introduced    datetime64[ns]
PRODUCT_PRODUCT_TYPE_code               string[python]
PRODUCT_PRODUCTION_COST_cost                   float64
PRODUCT_MARGIN_margin                          float64
PRODUCT_image_image                     string[python]
PRODUCT_LANGUAGE_language               string[python]
PRODUCT_name_product                    string[python]
PRODUCT_description_description         string[python]
PRODUCT_PRODUCT_LINE_code               string[python]
PRODUCT_PRODUCT_TYPE_code_en            string[python]
PRODUCT_PRODUCT_LINE_code_en            string[python]
RETAILER_c

### Training

In [47]:
training['TRAINING_SALES_STAFF_CODE'] = training['TRAINING_SALES_STAFF_CODE'].astype('int')
training['TRAINING_COURSE_CODE'] = training['TRAINING_COURSE_CODE'].astype('int')
training['TRAINING_YEAR'] = training['TRAINING_YEAR'].astype('int')
training['COURSE_code'] = training['COURSE_code'].astype('int')
training['COURSE_DESCRIPTION'] = training['COURSE_DESCRIPTION'].astype('string')

training.dtypes

TRAINING_YEAR                         int32
TRAINING_SALES_STAFF_CODE             int32
TRAINING_COURSE_CODE                  int32
index                                 int64
COURSE_code                           int32
COURSE_DESCRIPTION           string[python]
dtype: object

### Satisfaction

In [48]:
satisfaction['SATISFACTION_SALES_STAFF_CODE'] = satisfaction['SATISFACTION_SALES_STAFF_CODE'].astype('int')
satisfaction['SATISFACTION_SATISFACTION_TYPE_CODE'] = satisfaction['SATISFACTION_SATISFACTION_TYPE_CODE'].astype('int')
satisfaction['SATISFACTION_YEAR'] = satisfaction['SATISFACTION_YEAR'].astype('int')
satisfaction['SATISFACTION_TYPE_code'] = satisfaction['SATISFACTION_TYPE_code'].astype('int')
satisfaction['SATISFACTION_TYPE_DESCRIPTION'] = satisfaction['SATISFACTION_TYPE_DESCRIPTION'].astype('string')

satisfaction.dtypes

SATISFACTION_YEAR                               int32
SATISFACTION_SALES_STAFF_CODE                   int32
SATISFACTION_SATISFACTION_TYPE_CODE             int32
index                                           int64
SATISFACTION_TYPE_code                          int32
SATISFACTION_TYPE_DESCRIPTION          string[python]
dtype: object

### Order_header

In [49]:
order_header['ORDER_HEADER_number'] = order_header['ORDER_HEADER_number'].astype('int')
order_header['ORDER_HEADER_RETAILER_NAME'] = order_header['ORDER_HEADER_RETAILER_NAME'].astype('string')
order_header['ORDER_HEADER_SALES_STAFF'] = order_header['ORDER_HEADER_SALES_STAFF'].astype('int')
order_header['ORDER_HEADER_SALES_BRANCH_CODE'] = order_header['ORDER_HEADER_SALES_BRANCH_CODE'].astype('int')
order_header['ORDER_HEADER_RETAILER_SITE_CODE'] = order_header['ORDER_HEADER_RETAILER_SITE_CODE'].astype('int')
order_header['ORDER_HEADER_RETAILER_CONTACT_CODE'] = order_header['ORDER_HEADER_RETAILER_CONTACT_CODE'].astype('int')
order_header['ORDER_METHOD_CODE'] = order_header['ORDER_METHOD_CODE'].astype('int')
order_header['RETAILER_CODE'] = order_header['RETAILER_code'].astype('int')
order_header['RETAILER_COMPANY_CODE_MR_company'] = order_header['RETAILER_COMPANY_CODE_MR_company'].astype('string')

order_header.dtypes

ORDER_HEADER_number                                              int32
ORDER_HEADER_RETAILER_NAME                              string[python]
ORDER_HEADER_RETAILER_SITE_CODE                                  int32
ORDER_HEADER_RETAILER_CONTACT_CODE                               int32
ORDER_HEADER_SALES_STAFF                                         int32
ORDER_HEADER_SALES_BRANCH_CODE                                   int32
ORDER_HEADER_ORDER_DATE                                 datetime64[ns]
ORDER_METHOD_CODE                                                int32
RETAILER_code                                                   object
RETAILER_COMPANY_CODE_MR_company                        string[python]
RETAILER_name                                                   object
RETAILER_RETAILER_TYPE_retailer_type_code                       object
RETAILER_RETAILER_TYPE_retailer_type_EN                         object
index                                                            int64
SALES_

### Retailer_segment

In [50]:
retailer_segment['RETAILER_SEGMENT_segment_code'] = retailer_segment['RETAILER_SEGMENT_segment_code'].astype('int')
retailer_segment['RETAILER_SEGMENT_language'] = retailer_segment['RETAILER_SEGMENT_language'].astype('string')
retailer_segment['RETAILER_SEGMENT_segment_name'] = retailer_segment['RETAILER_SEGMENT_segment_name'].astype('string')
retailer_segment['RETAILER_SEGMENT_SEGMENT_DESCRIPTION_description'] = retailer_segment['RETAILER_SEGMENT_SEGMENT_DESCRIPTION_description'].astype('string')

retailer_segment.dtypes

RETAILER_SEGMENT_segment_code                                int32
RETAILER_SEGMENT_language                           string[python]
RETAILER_SEGMENT_segment_name                       string[python]
RETAILER_SEGMENT_SEGMENT_DESCRIPTION_description    string[python]
dtype: object

### Retailer_headquarters

In [51]:
retailer_headquarters['RETAILER_HEADQUARTER_codemr'] = retailer_headquarters['RETAILER_HEADQUARTER_codemr'].astype('int')
retailer_headquarters['RETAIL_HEADQUARTER_retailer_name'] = retailer_headquarters['RETAIL_HEADQUARTER_retailer_name'].astype('string')
retailer_headquarters['RETAILER_HEADQUARTER_address1_address'] = retailer_headquarters['RETAILER_HEADQUARTER_address1_address'].astype('string')
retailer_headquarters['RETAILER_HEADQUARTER_address2_address'] = retailer_headquarters['RETAILER_HEADQUARTER_address2_address'].astype('string')
retailer_headquarters['RETAILER_HEADQUARTER_city_city'] = retailer_headquarters['RETAILER_HEADQUARTER_city_city'].astype('string')
retailer_headquarters['RETAILER_HEADQUARTER_region_region'] = retailer_headquarters['RETAILER_HEADQUARTER_region_region'].astype('string')
retailer_headquarters['RETAILER_HEADQUARTER_postal_zone_postal_zone'] = retailer_headquarters['RETAILER_HEADQUARTER_postal_zone_postal_zone'].astype('string')
retailer_headquarters['RETAILER_HEADQUARTER_country_code_country'] = retailer_headquarters['RETAILER_HEADQUARTER_country_code_country'].astype('string')
retailer_headquarters['RETAILER_HEADQUARTER_phone_phone'] = retailer_headquarters['RETAILER_HEADQUARTER_phone_phone'].astype('string')
retailer_headquarters['RETAILER_HEADQUARTER_fax_fax'] = retailer_headquarters['RETAILER_HEADQUARTER_fax_fax'].astype('string')
retailer_headquarters['RETAILER_HEADQUARTER_segment_code'] = retailer_headquarters['RETAILER_HEADQUARTER_segment_code'].astype('int')

retailer_headquarters.dtypes

RETAILER_HEADQUARTER_codemr                              int32
RETAIL_HEADQUARTER_retailer_name                string[python]
RETAILER_HEADQUARTER_address1_address           string[python]
RETAILER_HEADQUARTER_address2_address           string[python]
RETAILER_HEADQUARTER_city_city                  string[python]
RETAILER_HEADQUARTER_region_region              string[python]
RETAILER_HEADQUARTER_postal_zone_postal_zone    string[python]
RETAILER_HEADQUARTER_country_code_country       string[python]
RETAILER_HEADQUARTER_phone_phone                string[python]
RETAILER_HEADQUARTER_fax_fax                    string[python]
RETAILER_HEADQUARTER_segment_code                        int32
dtype: object

# Toevoegen van nieuwe kolommen
Toevoegen van de afgeleide informatie die ook in het ETL en de data warehouse moet komen.
Full name is al gedaan via sql

In [52]:
sales_target_data['SALES_TARGET_DATA_TARGET_COST'] = sales_target_data['SALES_TARGETDATA_SALES_TARGET'] * sales_target_data['PRODUCT_PRODUCTION_COST_cost']

sales_target_data['SALES_TARGET_DATA_TARGET_MARGIN'] = sales_target_data['SALES_TARGETDATA_SALES_TARGET'] * sales_target_data['PRODUCT_MARGIN_margin']

order_details['ORDER_DETAILS_TOTAL_COST'] = order_details['ORDER_DETAILS_QUANTITY_quantity'] * order_details['UNIT_COST_cost']

order_details['ORDER_DETAILS_TOTAL_MARGIN'] = order_details['ORDER_DETAILS_QUANTITY_quantity'] * order_details['UNIT_SALE_sale']

go_sales_product_forecast['GO_SALES_PRODUCT_FORECAST_EXPECTED_COST'] = go_sales_product_forecast['GO_SALES_PRODUCT_FORECAST_EXPECTED_VOLUME'] * product['PRODUCT_PRODUCTION_COST_cost']

go_sales_product_forecast['GO_SALES_PRODUCT_FORECAST_EXPECTED_MARGIN'] = go_sales_product_forecast['GO_SALES_PRODUCT_FORECAST_EXPECTED_VOLUME'] * product['PRODUCT_MARGIN_margin']

# Dimensies
product['MINIMUM_SALE_PRICE'] = product['PRODUCT_PRODUCTION_COST_cost'] + product['PRODUCT_MARGIN_margin']
returned_item['RETURNED_ITEMS_TOTAL_PRICE'] = returned_item['RETURNED_ITEM_QUANTITY_quantity'] * order_details['UNIT_SALE_sale']
course['COURSE_DESCRIPTION_SHORT'] = None
retailer_segment['SEGMENT_DESCRIPTION_SHORT'] = None
satisfaction_type['SATISFACTION_TYPE_DESCRIPTION_SHORT'] = None

# if address2 is empty, main_address should be address1 and vice versa. If both addresses are available, main_address should be null
sales_branch['MAIN_ADDRESS'] = np.where(sales_branch['SALES_BRANCH_ADDRESS2_address'].isna(), sales_branch['SALES_BRANCH_ADDRESS1_address'], np.where(sales_branch['SALES_BRANCH_ADDRESS1_address'].isna(), sales_branch['SALES_BRANCH_ADDRESS2_address'], ''))

retailer_site['MAIN_ADDRESS'] = np.where(retailer_site['RETAILER_SITE_ADDRESS2_address'].isna(), retailer_site['RETAILER_SITE_ADDRESS1_address'], np.where(retailer_site['RETAILER_SITE_ADDRESS1_address'].isna(), retailer_site['RETAILER_SITE_ADDRESS2_address'], ''))

retailer_headquarters['MAIN_ADDRESS'] = np.where(retailer_headquarters['RETAILER_HEADQUARTER_address2_address'].isna(), retailer_headquarters['RETAILER_HEADQUARTER_address1_address'], np.where(retailer_headquarters['RETAILER_HEADQUARTER_address1_address'].isna(), retailer_headquarters['RETAILER_HEADQUARTER_address2_address'], ''))


In [53]:
product.head()

,PRODUCT_number,PRODUCT_INTRODUCTION_DATE_introduced,PRODUCT_PRODUCT_TYPE_code,PRODUCT_PRODUCTION_COST_cost,PRODUCT_MARGIN_margin,PRODUCT_image_image,PRODUCT_LANGUAGE_language,PRODUCT_name_product,PRODUCT_description_description,PRODUCT_PRODUCT_LINE_code,PRODUCT_PRODUCT_TYPE_code_en,PRODUCT_PRODUCT_LINE_code_en,MINIMUM_SALE_PRICE
0,1,2011-02-15,1,4.00,0.33,P01CE1CG1.jpg,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",1,Cooking Gear,Camping Equipment,4.33
1,10,2011-02-15,1,10.00,0.40,P10CE1CG1.jpg,EN,TrailChef Utensils,"Spoon, fork and knife set made of a light yet ...",1,Cooking Gear,Camping Equipment,10.40
2,100,2011-02-15,17,3.00,0.50,P100OP4FA17.jpg,EN,Insect Bite Relief,The Insect Bite Relief helps the itching and s...,4,First Aid,Outdoor Protection,3.50
3,101,2019-12-15,18,305.54,0.43,P101GE5IR18.jpg,EN,Hailstorm Steel Irons,Iron is 17-4 stainless steel. Shafts are grap...,5,Irons,Golf Equipment,305.97
4,102,2019-10-12,18,380.95,0.51,P102GE5IR18.jpg,EN,Hailstorm Titanium Irons,Made entirely of pure titanium. The ultimate i...,5,Irons,Golf Equipment,381.46


## Opslaan historische data
Aan alle tabellen een kolom toevoegen met de datum van de wijziging en of dit de huidige versie is.

In [54]:
for df in all_dataframes:
    df['LAST_UPDATED'] = pd.to_datetime('today')
    df['CURRENT'] = 1

# Data opslaan in de database
De pandas dataframes opslaan in de mssql database

## Unit
works

In [55]:
print(unit.shape[0])
for index, row in unit.iterrows():
    query = 'INSERT INTO Unit (UNIT_id, UNIT_COST_cost, UNIT_PRICE_price, UNIT_SALE_sale) VALUES (?,?,?,?)'
    values = (row['UNIT_id'], row['UNIT_COST_cost'], row['UNIT_PRICE_price'], row['UNIT_SALE_sale'])
    export_cursor.execute(query, values)

43063


## Sales_staff
error related to main adress

In [56]:
print(sales_staff.shape[0])
print(sales_staff.dtypes.iloc[11])
for index, row in sales_staff.iterrows():
    query = 'INSERT INTO Sales_staff (SALES_STAFF_code, SALES_STAFF_email, SALES_STAFF_extension, SALES_STAFF_POSITION_EN_position, SALES_STAFF_WORK_PHONE_work_phone, SALES_STAFF_DATE_HIRED_hired, SALES_STAFF_MANAGER_CODE_manager, SALES_STAFF_FAX_fax, SALES_STAFF_FIRST_NAME_first_name, SALES_STAFF_LAST_NAME_last_name, SALES_STAFF_SALES_BRANCH_CODE_branch_code, SALES_STAFF_SALES_BRANCH_ADDRESS_address, SALES_STAFF_SALES_BRANCH_ADDRESS_EXTRA_address_extra) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)'

    values = (row['SALES_STAFF_code'], row['SALES_STAFF_email'], row['SALES_STAFF_extension'], row['SALES_STAFF_POSITION_EN_position'], row['SALES_STAFF_WORK_PHONE_work_phone'], row['SALES_STAFF_DATE_HIRED_hired'], row['SALES_STAFF_MANAGER_CODE_manager'], row['SALES_STAFF_FAX_fax'], row['SALES_STAFF_FIRST_NAME_first_name'], row['SALES_STAFF_LAST_NAME_last_name'], row['SALES_STAFF_SALES_BRANCH_CODE_branch_code'], row['SALES_STAFF_SALES_BRANCH_ADDRESS_address'], row['SALES_STAFF_SALES_BRANCH_ADDRESS_EXTRA_address_extra'])

    export_cursor.execute(query, values)

102
string


ProgrammingError: ('Invalid parameter type.  param-index=12 param-type=NAType', 'HY105')

## satisfaction_type
works

In [57]:
print(satisfaction_type.shape[0])
for index, row in satisfaction_type.iterrows():
    query = 'INSERT INTO Satisfaction_type (SATISFACTION_TYPE_code, SATISFACTION_TYPE_DESCRIPTION, CURRENT_VALUE) VALUES (?,?,?)'
    values = (row['SATISFACTION_TYPE_code'], row['SATISFACTION_TYPE_DESCRIPTION'], 1)
    export_cursor.execute(query, values)

5


## Course
works

In [58]:
print(course.shape[0])
for index, row in course.iterrows():
    query = 'INSERT INTO Course (COURSE_code, COURSE_DESCRIPTION, CURRENT_VALUE) VALUES (?,?,?)'
    values = (row['COURSE_code'], row['COURSE_DESCRIPTION'], 1)
    export_cursor.execute(query, values)

9


## year
duplicate error

In [59]:
print(year.shape[0])
for index, row in year.iterrows():
    query = 'INSERT INTO Year (YEAR) VALUES (?)'
    values = index
    export_cursor.execute(query, values)

214


IntegrityError: ('23000', "[23000] [Microsoft][ODBC SQL Server Driver][SQL Server]Violation of PRIMARY KEY constraint 'PK__Year__D4BD6055BF51AF4B'. Cannot insert duplicate key in object 'dbo.Year'. The duplicate key value is (2004). (2627) (SQLExecDirectW); [23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The statement has been terminated. (3621)")

## date
works

In [60]:
print(date.shape[0])
for index, row in date.iterrows():
    query = 'INSERT INTO Date (DATE_date) VALUES (?)'
    values = index
    export_cursor.execute(query, values)

76702


## order
works

In [61]:
print(order.shape[0])
for index, row in order.iterrows():
    query = 'INSERT INTO "Order" (ORDER_order_number, ORDER_ORDER_METHOD_CODE_method_code, ORDER_ORDER_METHOD_EN_method) VALUES (?,?,?)'
    values = (index,row['ORDER_ORDER_METHOD_CODE_method_code'], row['ORDER_ORDER_METHOD_EN_method'])
    export_cursor.execute(query, values)

7


## retailer_site
error related to adress 2

In [62]:
print(retailer_site.shape[0])
for index, row in retailer_site.iterrows():
    query = 'INSERT INTO Retailer_site (RETAILER_SITE_code, RETAILER_SITE_COUNTRY_CODE_country, RETAILER_SITE_CITY_city, RETAILER_SITE_REGION_region, RETAILER_SITE_POSTAL_ZONE_postal_zone, RETAILER_SITE_RETAILER_CODE_retailer_code, RETAILER_SITE_ACTIVE_INDICATOR_indicator, RETAILER_SITE_ADDRESS1_address, RETAILER_SITE_ADDRESS2_address) VALUES (?,?,?,?,?,?,?,?,?)'
    values = (row['RETAILER_SITE_code'], row['RETAILER_SITE_COUNTRY_CODE_country'], row['RETAILER_SITE_CITY_city'], row['RETAILER_SITE_REGION_region'], row['RETAILER_SITE_POSTAL_ZONE_postal_zone'], row['RETAILER_SITE_RETAILER_CODE_retailer_code'], row['RETAILER_SITE_ACTIVE_INDICATOR_indicator'], row['RETAILER_SITE_ADDRESS1_address'], row['RETAILER_SITE_ADDRESS2_address'])
    export_cursor.execute(query, values)

391


ProgrammingError: ('Invalid parameter type.  param-index=8 param-type=NAType', 'HY105')

## sales_branch
error related to country

In [63]:
print(sales_branch.shape[0])
for index, row in sales_branch.iterrows():
    query = 'INSERT INTO Sales_branch (SALES_BRANCH_code, SALES_BRANCH_COUNTRY_CODE_country, SALES_BRANCH_REGION_region, SALES_BRANCH_CITY_city, SALES_BRANCH_POSTAL_ZONE_postal_zone, SALES_BRANCH_ADDRESS1_address, SALES_BRANCH_ADDRESS2_address, CURRENT_VALUE) VALUES (?,?,?,?,?,?,?)'
    values = (row['SALES_BRANCH_code'], row['SALES_BRANCH_COUNTRY_CODE_country'], row['SALES_BRANCH_REGION_region'], row['SALES_BRANCH_CITY_city'], row['SALES_BRANCH_POSTAL_ZONE_postal_zone'], row['SALES_BRANCH_ADDRESS1_address'], row['SALES_BRANCH_ADDRESS2_address'])
    export_cursor.execute(query, values)

28


ProgrammingError: ('Invalid parameter type.  param-index=2 param-type=NAType', 'HY105')

## retailer_contact
foreign key error (retailer_site_code)

In [64]:
print(retailer_contact.shape[0])
for index, row in retailer_contact.iterrows():
    query = 'INSERT INTO Retailer_contact (RETAILER_CONTACT_code, RETAILER_CONTACT_email, RETAILER_CONTACT_RETAILER_SITE_CODE_site_code, RETAILER_CONTACT_JOB_POSITION_EN_position, RETAILER_CONTACT_EXTENSION_extension, RETAILER_CONTACT_FAX_fax, RETAILER_CONTACT_FIRST_NAME_first_name, RETAILER_CONTACT_LAST_NAME_last_name) VALUES (?,?,?,?,?,?,?,?)'
    
    values = (row['RETAILER_CONTACT_code'], row['RETAILER_CONTACT_email'], row['RETAILER_CONTACT_RETAILER_SITE_CODE_site_code'], row['RETAILER_CONTACT_JOB_POSITION_EN_position'], row['RETAILER_CONTACT_EXTENSION_extension'], row['RETAILER_CONTACT_FAX_fax'], row['RETAILER_CONTACT_FIRST_NAME_first_name'], row['RETAILER_CONTACT_LAST_NAME_last_name'])
    
    export_cursor.execute(query, values)

391


IntegrityError: ('23000', '[23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The INSERT statement conflicted with the FOREIGN KEY constraint "FK__Retailer___RETAI__68487DD7". The conflict occurred in database "DEDS_DataWarehouse", table "dbo.Retailer_site", column \'RETAILER_SITE_code\'. (547) (SQLExecDirectW); [23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The statement has been terminated. (3621)')

## retailer
na error in code_mr_company

In [65]:
print(retailer.shape[0])
for index, row in retailer.iterrows():
    query = 'INSERT INTO Retailer (RETAILER_code, RETAILER_name, RETAILER_COMPANY_CODE_MR_company, RETAILER_RETAILER_TYPE_retailer_type_code, RETAILER_RETAILER_TYPE_retailer_type_EN, CURRENT_VALUE) VALUES (?,?,?,?,?,?)'
    values = (row['RETAILER_code'], row['RETAILER_name'], row['RETAILER_COMPANY_CODE_MR_company'], row['RETAILER_RETAILER_TYPE_retailer_type_code'], row['RETAILER_RETAILER_TYPE_retailer_type_EN'], 1)
    export_cursor.execute(query, values)

109


ProgrammingError: ('Invalid parameter type.  param-index=2 param-type=NAType', 'HY105')

## product
truncation error (line code is too short?)

In [66]:
print(product.shape[0])
for index, row in product.iterrows():
    query = 'INSERT INTO Product (PRODUCT_number, PRODUCT_name_product, PRODUCT_description_description, PRODUCT_image_image, PRODUCT_INTRODUCTION_DATE_introduced, PRODUCT_PRODUCTION_COST_cost, PRODUCT_MARGIN_margin, PRODUCT_LANGUAGE_language, PRODUCT_PRODUCT_LINE_code, PRODUCT_PRODUCT_LINE_code_en, PRODUCT_PRODUCT_TYPE_code, PRODUCT_PRODUCT_TYPE_code_en) VALUES (?,?,?,?,?,?,?,?,?,?,?,?)'
    values = (row['PRODUCT_number'], row['PRODUCT_name_product'], row['PRODUCT_description_description'], row['PRODUCT_image_image'], row['PRODUCT_INTRODUCTION_DATE_introduced'], row['PRODUCT_PRODUCTION_COST_cost'], row['PRODUCT_MARGIN_margin'], row['PRODUCT_LANGUAGE_language'], row['PRODUCT_PRODUCT_LINE_code'], row['PRODUCT_PRODUCT_LINE_code_en'], row['PRODUCT_PRODUCT_TYPE_code'], row['PRODUCT_PRODUCT_TYPE_code_en'])
    export_cursor.execute(query, values)

115


IntegrityError: ('23000', "[23000] [Microsoft][ODBC SQL Server Driver][SQL Server]Cannot insert the value NULL into column 'PRODUCT_MINIMUM_SALE_PRICE_minPrice', table 'DEDS_DataWarehouse.dbo.Product'; column does not allow nulls. INSERT fails. (515) (SQLExecDirectW); [23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The statement has been terminated. (3621)")

## order_details
error in column names

In [67]:
print(order_details.shape[0])
for index, row in order_details.iterrows():
    query = 'INSERT INTO Order_details (ORDER_DETAILS_code, ORDER_DETAILS_QUANTITY_quantity, ORDER_DETAILS_TOTAL_COST_total, ORDER_DETAILS_TOTAL_MARGIN_margin, ORDER_DETAILS_RETURN_CODE_returned, ORDER_DETAILS_ORDER_NUMBER_order, ORDER_DETAILS_PRODUCT_NUMBER_product, ORDER_DETAILS_UNIT_ID_unit) VALUES (?,?,?,?,?,?,?,?)'
    values = (row['ORDER_DETAILS_code'], row['ORDER_DETAILS_QUANTITY_quantity'], row['ORDER_DETAILS_TOTAL_COST_total'], row['ORDER_DETAILS_TOTAL_MARGIN_margin'], row['ORDER_DETAILS_RETURN_CODE_returned'], row['ORDER_DETAILS_ORDER_NUMBER_order'], row['ORDER_DETAILS_PRODUCT_NUMBER_product'], row['ORDER_DETAILS_UNIT_ID_unit'])
    export_cursor.execute(query, values)

706


KeyError: 'ORDER_DETAILS_code'

## returned_item
column name error

In [68]:
print(returned_item.shapqe[0])
for index, row in returned_item.iterrows():
    query = 'INSERT INTO Returned_item (RETURNED_ITEM_code, RETURNED_ITEM_DATE, RETURNED_ITEM_QUANTITY, RETURNED_ITEM_ORDER_DETAIL_CODE, RETURNED_ITEM_RETURN_REASON_code, RETURNED_ITEM_RETURN_REASON_description_en, RETURNED_ITEM_RETURNED_ITEMS_TOTAL_PRICE) VALUES (?,?,?,?,?,?,?)'
    values = (row['RETURNED_ITEM_code'], row['RETURNED_ITEM_DATE'], row['RETURNED_ITEM_QUANTITY'], row['RETURNED_ITEM_ORDER_DETAIL_CODE'], row['RETURNED_ITEM_RETURN_REASON_code'], row['RETURNED_ITEM_RETURN_REASON_description_en'], row['RETURNED_ITEM_RETURNED_ITEMS_TOTAL_PRICE'])
    export_cursor.execute(query, values)

706


KeyError: 'RETURNED_ITEM_DATE'

## sales_target_data
KeyError: 'SALES_TARGETDATA_TARGET_COST'

In [76]:
print(sales_target_data.shape[0])
for index, row in sales_target_data.iterrows():
    query = 'INSERT INTO SALES_TARGETDATA (SALES_TARGETDATA_SALES_YEAR, SALES_TARGETDATA_SALES_PERIOD, SALES_TARGETDATA_RETAILER_NAME, SALES_TARGETDATA_SALES_TARGET, SALES_TARGETDATA_TARGET_COST, SALES_TARGETDATA_TARGET_MARGIN, SALES_TARGETDATA_SALES_STAFF_CODE, SALES_TARGETDATA_PRODUCT_NUMBER, SALES_TARGETDATA_RETAILER_CODE) VALUES (?,?,?,?,?,?,?,?,?)'
    values = (row['SALES_TARGETDATA_SALES_YEAR'], row['SALES_TARGETDATA_SALES_PERIOD'], row['SALES_TARGETDATA_RETAILER_NAME'], row['SALES_TARGETDATA_SALES_TARGET'], row['SALES_TARGET_DATA_TARGET_COST'], row['SALES_TARGET_DATA_TARGET_MARGIN'], row['SALES_TARGETDATA_SALES_STAFF_CODE'], row['PRODUCT_number'], row['RETAILER_code'])
    export_cursor.execute(query, values)

39530


IntegrityError: ('23000', "[23000] [Microsoft][ODBC SQL Server Driver][SQL Server]Cannot insert the value NULL into column 'SALES_TARGETDATA_id', table 'DEDS_DataWarehouse.dbo.SALES_TARGETDATA'; column does not allow nulls. INSERT fails. (515) (SQLExecDirectW); [23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The statement has been terminated. (3621)")

## training
foreign key constraint, sales_staff

In [77]:
print(training.shape[0])
for index, row in training.iterrows():
    query = 'INSERT INTO Training (TRAINING_SALES_STAFF_CODE, TRAINING_COURSE_CODE, TRAINING_YEAR) VALUES (?,?,?)'
    values = (row['TRAINING_SALES_STAFF_CODE'], row['TRAINING_COURSE_CODE'], row['TRAINING_YEAR'])
    export_cursor.execute(query, values)

402


IntegrityError: ('23000', '[23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The INSERT statement conflicted with the FOREIGN KEY constraint "FK__Training__TRAINI__0A9D95DB". The conflict occurred in database "DEDS_DataWarehouse", table "dbo.Sales_staff", column \'SALES_STAFF_code\'. (547) (SQLExecDirectW); [23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The statement has been terminated. (3621)')

## satisfaction
foreign key constraint, sales_staff

In [78]:
print(satisfaction.shape[0])
for index, row in satisfaction.iterrows():
    query = 'INSERT INTO Satisfaction (SATISFACTION_SALES_STAFF_CODE, SATISFACTION_SATISFACTION_TYPE_CODE, SATISFACTION_YEAR) VALUES (?,?,?)'
    values = (row['SATISFACTION_SALES_STAFF_CODE'], row['SATISFACTION_SATISFACTION_TYPE_CODE'], row['SATISFACTION_YEAR'])
    export_cursor.execute(query, values)

301


IntegrityError: ('23000', '[23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The INSERT statement conflicted with the FOREIGN KEY constraint "FK__Satisfact__SATIS__114A936A". The conflict occurred in database "DEDS_DataWarehouse", table "dbo.Sales_staff", column \'SALES_STAFF_code\'. (547) (SQLExecDirectW); [23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The statement has been terminated. (3621)')

## order_header
key errors

In [79]:
print(order_header.shape[0])
for index, row in order_header.iterrows():
    query = 'INSERT INTO Order_header (ORDER_HEADER_number, ORDER_HEADER_SALES_STAFF_CODE, ORDER_HEADER_SALES_BRANCH_CODE, ORDER_HEADER_RETAILER_SITE_CODE, ORDER_HEADER_RETAILER_CONTACT_CODE, ORDER_HEADER_RETAILER_CODE, ORDER_DATE) VALUES (?,?,?,?,?,?,?)'
    values = (row['ORDER_HEADER_number'], row['ORDER_HEADER_SALES_STAFF_CODE'], row['ORDER_HEADER_SALES_BRANCH_CODE'], row['ORDER_HEADER_RETAILER_SITE_CODE'], row['ORDER_HEADER_RETAILER_CONTACT_CODE'], row['ORDER_HEADER_RETAILER_CODE'], row['ORDER_DATE'])
    export_cursor.execute(query, values)

4965


KeyError: 'ORDER_HEADER_SALES_STAFF_CODE'

## retailer_segment
works

In [80]:
print(retailer_segment.shape[0])
for index, row in retailer_segment.iterrows():
    query = 'INSERT INTO Retailer_segment (RETAILER_SEGMENT_segment_code, RETAILER_SEGMENT_language, RETAILER_SEGMENT_segment_name, RETAILER_SEGMENT_SEGMENT_DESCRIPTION_description, SEGMENT_DESCRIPTION_description_short, CURRENT_VALUE) VALUES (?,?,?,?,?,?)'
    values = (row['RETAILER_SEGMENT_segment_code'], row['RETAILER_SEGMENT_language'], row['RETAILER_SEGMENT_segment_name'], row['RETAILER_SEGMENT_SEGMENT_DESCRIPTION_description'], row['SEGMENT_DESCRIPTION_SHORT'], 1)
    export_cursor.execute(query, values)

12


## retailer_headquarters
na problem with adres 2

In [81]:
print(retailer_headquarters.shape[0])
for index, row in retailer_headquarters.iterrows():
    query = 'INSERT INTO Retailer_headquarters (RETAILER_HEADQUARTER_codemr, RETAIL_HEADQUARTER_retailer_name, RETAILER_HEADQUARTER_address1_address, RETAILER_HEADQUARTER_address2_address, RETAILER_HEADQUARTER_city_city, RETAILER_HEADQUARTER_region_region, RETAILER_HEADQUARTER_postal_zone_postal_zone, RETAILER_HEADQUARTER_country_code_country, RETAILER_HEADQUARTER_phone_phone, RETAILER_HEADQUARTER_fax_fax, RETAILER_HEADQUARTER_segment_code, MAIN_ADDRESS, CURRENT_VALUE) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)'
    values = (row['RETAILER_HEADQUARTER_codemr'], row['RETAIL_HEADQUARTER_retailer_name'], row['RETAILER_HEADQUARTER_address1_address'], row['RETAILER_HEADQUARTER_address2_address'], row['RETAILER_HEADQUARTER_city_city'], row['RETAILER_HEADQUARTER_region_region'], row['RETAILER_HEADQUARTER_postal_zone_postal_zone'], row['RETAILER_HEADQUARTER_country_code_country'], row['RETAILER_HEADQUARTER_phone_phone'], row['RETAILER_HEADQUARTER_fax_fax'], row['RETAILER_HEADQUARTER_segment_code'], row['MAIN_ADDRESS'], 1)
    export_cursor.execute(query, values)

414


ProgrammingError: ('Invalid parameter type.  param-index=3 param-type=NAType', 'HY105')

## Go_sales_inventory_levels
foreign key constraint, product

In [82]:
print(go_sales_inventory_levels.shape[0])
for index, row in go_sales_inventory_levels.iterrows():
    query = 'INSERT INTO GO_SALES_INVENTORY_LEVELS (GO_SALES_INVENTORY_LEVELS_PRODUCT_NUMBER, GO_SALES_INVENTORY_LEVELS_YEAR_MONTH, GO_SALES_INVENTORY_LEVELS_INVENTORY_COUNT) VALUES (?,?,?)'
    values = (row['GO_SALES_INVENTORY_LEVELS_PRODUCT_NUMBER'], row['GO_SALES_INVENTORY_LEVELS_YEAR_MONTH'], row['GO_SALES_INVENTORY_LEVELS_INVENTORY_COUNT'])
    export_cursor.execute(query, values)

3888


IntegrityError: ('23000', '[23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The INSERT statement conflicted with the FOREIGN KEY constraint "FK__GO_SALES___GO_SA__208CD6FA". The conflict occurred in database "DEDS_DataWarehouse", table "dbo.Product", column \'PRODUCT_number\'. (547) (SQLExecDirectW); [23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The statement has been terminated. (3621)')

## Go_sales_product_forecast
foreign key constraint, product

In [83]:
print(go_sales_product_forecast.shape[0])
for index, row in go_sales_product_forecast.iterrows():
    query = 'INSERT INTO GO_SALES_PRODUCT_FORECAST (GO_SALES_PRODUCT_FORECAST_PRODUCT_NUMBER, GO_SALES_PRODUCT_FORECAST_YEAR_MONTH, GO_SALES_PRODUCT_FORECAST_EXPECTED_VOLUME, GO_SALES_PRODUCT_FORECAST_EXPECTED_COST, GO_SALES_PRODUCT_FORECAST_EXPECTED_MARGIN) VALUES (?,?,?,?,?)'
    values = (row['GO_SALES_PRODUCT_FORECAST_PRODUCT_NUMBER'], row['GO_SALES_PRODUCT_FORECAST_YEAR_MONTH'], row['GO_SALES_PRODUCT_FORECAST_EXPECTED_VOLUME'], row['GO_SALES_PRODUCT_FORECAST_EXPECTED_COST'], row['GO_SALES_PRODUCT_FORECAST_EXPECTED_MARGIN'])
    export_cursor.execute(query, values)

3872


IntegrityError: ('23000', '[23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The INSERT statement conflicted with the FOREIGN KEY constraint "FK__GO_SALES___GO_SA__25518C17". The conflict occurred in database "DEDS_DataWarehouse", table "dbo.Product", column \'PRODUCT_number\'. (547) (SQLExecDirectW); [23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The statement has been terminated. (3621)')

## clean up

In [ ]:
# close cursor
export_conn.close()

# close connection
export_conn.close()